# 3.1 引言

> 来源：[3.1   Introduction](http://www-inst.eecs.berkeley.edu/~cs61a/sp12/book/interpretation.html#introduction)

> 译者：[飞龙](https://github.com/wizardforcel)

> 协议：[CC BY-NC-SA 4.0](http://creativecommons.org/licenses/by-nc-sa/4.0/)

第一章和第二章描述了编程的两个基本元素：数据和函数之间的紧密联系。我们看到了高阶函数如何将函数当做数据操作。我们也看到了数据可以使用消息传递和对象系统绑定行为。我们已经学到了组织大型程序的技巧，例如函数抽象，数据抽象，类的继承，以及泛用函数。这些核心概念构成了坚实的基础，来构建模块化，可维护和可扩展的程序。

这一章专注于编程的第三个基本元素：程序自身。Python 程序只是文本的集合。只有通过解释过程，我们才可以基于文本执行任何有意义的计算。类似 Python 的编程语言很实用，因为我们可以定义解释器，它是一个执行 Python 求值和执行过程的程序。把它看做编程中最基本的概念并不夸张。解释器只是另一个程序，它确定编程语言中表达式的意义。

接受这一概念，需要改变我们自己作为程序员的印象。我们需要将自己看做语言的设计者，而不只是由他人设计的语言用户。

## 3.1.1 编程语言

实际上，我们可以将许多程序看做一些语言的解释器。例如，上一章的约束传播器拥有自己的原语和组合方式。约束语言是十分专用的：它提供了一种声明式的方式来描述数学关系的特定种类，而不是一种用于描述计算的完全通用的语言。虽然我们已经设计了某种语言，这章的材料会极大扩展我们可解释的语言范围。

编程语言在语法结构、特性和应用领域上差别很大。在通用编程语言中，函数定义和函数调用的结构无处不在。另一方法，存在不包含对象系统、高阶函数或类似`while`和`for`语句的控制结构的强大的编程语言。为了展示语言可以有多么不同，我们会引入[Logo](http://www.cs.berkeley.edu/~bh/logo.html)作为强大并且具有表现力的编程语言的例子，它包含非常少的高级特性。

这一章中，我们会学习解释器的设计，以及在执行程序时，它们所创建的计算过程。为通用语言设计解释器的想法可能令人畏惧。毕竟，解释器是执行任何可能计算的程序，取决于它们的输入。但是，典型的解释器拥有简洁的通用结构：两个可变的递归函数，第一个求解环境中的表达式，第二个在参数上调用函数。

这些函数都是递归的，因为它们互相定义：调用函数需要求出函数体的表达式，而求出表达式可能涉及到调用一个或多个函数。这一章接下来的两节专注于递归函数和数据结构，它们是理解解释器设计的基础。这一章的结尾专注于两个新的编程语言，以及为其实现解释器的任务。

# 3.2 函数和所生成的过程

> 来源：[3.2   Functions and the Processes They Generate](http://www-inst.eecs.berkeley.edu/~cs61a/sp12/book/interpretation.html#functions-and-the-processes-they-generate)

> 译者：[飞龙](https://github.com/wizardforcel)

> 协议：[CC BY-NC-SA 4.0](http://creativecommons.org/licenses/by-nc-sa/4.0/)

函数是计算过程的局部演化模式。它规定了过程的每个阶段如何构建在之前的阶段之上。我们希望能够创建有关过程整体行为的语句，而过程的局部演化由一个或多个函数指定。这种分析通常非常困难，但是我们至少可以试图描述一些典型的过程演化模式。

在这一章中，我们会检测一些用于简单函数所生成过程的通用“模型”。我们也会研究这些过程消耗重要的计算资源，例如时间和空间的比例。

## 3.2.1 递归函数

如果函数的函数体直接或者间接自己调用自己，那么这个函数是递归的。也就是说，递归函数的执行过程可能需要再次调用这个函数。Python 中的递归函数不需要任何特殊的语法，但是它们的确需要一些注意来正确定义。

作为递归函数的介绍，我们以将英文单词转换为它的 Pig Latin 等价形式开始。Pig Latin 是一种隐语：对英文单词使用一种简单、确定的转换来掩盖单词的含义。Thomas Jefferson 据推测是[先行者](http://goo.gl/Cwysz)。英文单词的 Pig Latin 等价形式将辅音前缀（可能为空）从开头移动到末尾，并且添加`-ay`元音。所以，`pun`会变成`unpay`，`stout`会变成`outstay`，`all`会变成`allay`。

In [ ]:
def pig_latin(w):
        """Return the Pig Latin equivalent of English word w."""
        if starts_with_a_vowel(w):
            return w + 'ay'
        return pig_latin(w[1:] + w[0])
def starts_with_a_vowel(w):
        """Return whether w begins with a vowel."""
        return w[0].lower() in 'aeiou'

这个定义背后的想法是，一个以辅音开头的字符串的 Pig Latin 变体和另一个字符串的 Pig Latin 变体相同：它通过将第一个字母移到末尾来创建。于是，`sending`的 Pig Latin 变体就和`endings`的变体（`endingsay`）相同。`smother`的 Pig Latin 变体和`mothers`的变体（`othersmay`）相同。而且，将辅音从开头移动到末尾会产生带有更少辅音前缀的更简单的问题。在`sending`的例子中，将`s`移动到末尾会产生以元音开头的单词，我们的任务就完成了。

即使`pig_latin`函数在它的函数体中调用，`pig_latin`的定义是完整且正确的。

In [ ]:
pig_latin('pun')

能够基于函数自身来定义函数的想法可能十分令人混乱：“循环”定义如何有意义，这看起来不是很清楚，更不用说让计算机来执行定义好的过程。但是，我们能够准确理解递归函数如何使用我们的计算环境模型来成功调用。环境的图示和描述`pig_latin('pun')`求值的表达式树展示在下面：

![](img/pig_latin.png)

Python 求值过程的步骤产生如下结果：

1.  `pig_latin `的`def`语句 被执行，其中：
    1.  使用函数体创建新的`pig_latin`函数对象，并且
    2.  将名称`pig_latin`在当前（全局）帧中绑定到这个函数上。
2.  `starts_with_a_vowel `的`def`语句类似地执行。
3.  求出`pig_latin('pun')`的调用表达式，通过
    1.  求出运算符和操作数子表达式，通过
        1.  查找绑定到`pig_latin`函数的`pig_latin`名称
        2.  对字符串对象`'pun'`求出操作数字符串字面值
    2.  在参数`'pun'`上调用`pig_latin`函数，通过
        1.  添加扩展自全局帧的局部帧
        2.  将形参`w`绑定到当前帧的实参`'pun'`上。
        3.  在以当前帧起始的环境中执行`pig_latin`的函数体
            1.  最开始的条件语句没有效果，因为头部表达式求值为`False`
            2.  求出最后的返回表达式`pig_latin(w[1:] + w[0])`，通过
                1.  查找绑定到`pig_latin`函数的`pig_latin`名称
                2.  对字符串对象`'pun'`求出操作数表达式
                3.  在参数`'unp'`上调用`pig_latin`，它会从`pig_latin`函数体中的条件语句组返回预期结果。

就像这个例子所展示的那样，虽然递归函数具有循环特征，他仍旧正确调用。`pig_latin`函数调用了两次，但是每次都带有不同的参数。虽然第二个调用来自`pig_latin`自己的函数体，但由名称查找函数会成功，因为名称`pig_latin`在它的函数体执行前的环境中绑定。

这个例子也展示了 Python 的递归函数的求值过程如何与递归函数交互，来产生带有许多嵌套步骤的复杂计算过程，即使函数定义本身可能包含非常少的代码行数。

## 3.2.2 剖析递归函数

许多递归函数的函数体中都存在通用模式。函数体以基本条件开始，它是一个条件语句，为需要处理的最简单的输入定义函数行为。在`pig_latin`的例子中，基本条件对任何以元音开头的单词成立。这个时候，只需要返回末尾附加`ay`的参数。一些递归函数会有多重基本条件。

基本条件之后是一个或多个递归调用。递归调用有特定的特征：它们必须简化原始问题。在`pig_latin`的例子中，`w`中最开始辅音越多，就需要越多的处理工作。在递归调用`pig_latin(w[1:] + w[0])`中，我们在一个具有更少初始辅音的单词上调用`pig_latin` -- 这就是更简化的问题。每个成功的`pig_latin`调用都会更加简化，直到满足了基本条件：一个没有初始辅音的单词。

递归调用通过逐步简化问题来表达计算。与我们在过去使用过的迭代方式相比，它们通常以不同方式来解决问题。考虑用于计算`n`的阶乘的函数`fact`，其中`fact(4)`计算了`4! = 4·3·2·1 = 24`。

使用`while`语句的自然实现会通过将每个截至`n`的正数相乘来求出结果。

In [ ]:
def fact_iter(n):
        total, k = 1, 1
        while k <= n:
            total, k = total * k, k + 1
        return total

另一方面，阶乘的递归实现可以以`fact(n-1)`（一个更简单的问题）来表示`fact(n)`。递归的基本条件是问题的最简形式：`fact(1)`是`1`。

In [ ]:
def fact(n):
        if n == 1:
            return 1
        return n * fact(n-1)
fact(4)

函数的正确性可以轻易通过阶乘函数的标准数学定义来验证。

```
(n − 1)! = (n − 1)·(n − 2)· ... · 1
n！ = n·(n − 1)·(n − 2)· ... · 1
n! = n·(n − 1)!  
```

这两个阶乘函数在概念上不同。迭代的函数通过将每个式子，从基本条件`1`到最终的总数逐步相乘来构造结果。另一方面，递归函数直接从最终的式子`n`和简化的问题`fact(n-1)`构造结果。

将`fact`函数应用于更简单的问题实例，来展开递归的同时，结果最终由基本条件构建。下面的图示展示了递归如何向`fact`传入`1`而终止，以及每个调用的结果如何依赖于下一个调用，直到满足了基本条件。

![](img/fact.png)

虽然我们可以使用我们的计算模型展开递归，通常把递归调用看做函数抽象更清晰一些。也就是说，我们不应该关心`fact(n-1)`如何在`fact`的函数体中实现；我们只需要相信它计算了`n-1`的阶乘。将递归调用看做函数抽象叫做递归的“信仰飞跃”（leap of faith）。我们以函数自身来定义函数，但是仅仅相信更简单的情况在验证函数正确性时会正常工作。这个例子中我们相信，`fact(n-1)`会正确计算`(n-1)!`；我们只需要检查，如果满足假设`n!`是否正确计算。这样，递归函数正确性的验证就变成了一种归纳证明。

函数`fact_iter`和`fact`也不一样，因为前者必须引入两个额外的名称，`total`和`k`，它们在递归实现中并不需要。通常，迭代函数必须维护一些局部状态，它们会在计算过程中改变。在任何迭代的时间点上，状态刻画了已完成的结果，以及未完成的工作总量。例如，当`k`为`3`且`total`为`2`时，就还剩下两个式子没有处理，`3`和`4`。另一方面，`fact`由单一参数`n`来刻画。计算的状态完全包含在表达式树的结果中，它的返回值起到`total`的作用，并且在不同的帧中将`n`绑定到不同的值上，而不是显式跟踪`k`。

递归函数可以更加依赖于解释器本身，通过将计算状态储存为表达式树和环境的一部分，而不是显式使用局部帧中的名称。出于这个原因，递归函数通常易于定义，因为我们不需要试着弄清必须在迭代中维护的局部状态。另一方面，学会弄清由递归函数实现的计算过程，需要一些练习。

## 3.2.3 树形递归

另一个递归的普遍模式叫做树形递归。例如，考虑斐波那契序列的计算，其中每个数值都是前两个的和。

In [ ]:
def fib(n):
        if n == 1:
            return 0
        if n == 2:
            return 1
        return fib(n-2) + fib(n-1)
fib(6)

这个递归定义和我们之前的尝试有很大关系：它准确反映了斐波那契数的相似定义。考虑求出`fib(6)`所产生的计算模式，它展示在下面。为了计算`fib(6)`，我们需要计算`fib(5)`和`fib(4)`。为了计算`fib(5)`，我们需要计算`fib(4)`和`fib(3)`。通常，这个演化过程看起来像一棵树（下面的图并不是完整的表达式树，而是简化的过程描述；一个完整的表达式树也拥有同样的结构）。在遍历这棵树的过程中，每个蓝点都表示斐波那契数的已完成计算。

![](img/fib.png)

调用自身多次的函数叫做树形递归。以树形递归为原型编写的函数十分有用，但是用于计算斐波那契数则非常糟糕，因为它做了很多重复的计算。要注意整个`fib(4)`的计算是重复的，它几乎是一半的工作量。实际上，不难得出函数用于计算`fib(1)`和`fib(2)`（通常是树中的叶子数量）的时间是`fib(n+1)`。为了弄清楚这有多糟糕，我们可以证明`fib(n)`的值随着`n`以指数方式增长。所以，这个过程的步骤数量随输入以指数方式增长。


我们已经见过斐波那契数的迭代实现，出于便利在这里贴出来：

In [ ]:
def fib_iter(n):
        prev, curr = 1, 0  # curr is the first Fibonacci number.
        for _ in range(n-1):
             prev, curr = curr, prev + curr
        return curr

这里我们必须维护的状态由当前值和上一个斐波那契数组成。`for`语句也显式跟踪了迭代数量。这个定义并没有像递归方式那样清晰反映斐波那契数的数学定义。但是，迭代实现中所需的计算总数只是线性，而不是指数于`n`的。甚至对于`n`的较小值，这个差异都非常大。

然而我们不应该从这个差异总结出，树形递归的过程是没有用的。当我们考虑层次数据结构，而不是数值上的操作时，我们发现树形递归是自然而强大的工具。而且，树形过程可以变得更高效。

**记忆。**用于提升重复计算的递归函数效率的机制叫做记忆。记忆函数会为任何之前接受的参数储存返回值。`fib(4)`的第二次调用不会执行与第一次同样的复杂过程，而是直接返回第一次调用的已储存结果。

记忆函数可以自然表达为高阶函数，也可以用作装饰器。下面的定义为之前的已计算结果创建缓存，由被计算的参数索引。在这个实现中，这个字典的使用需要记忆函数的参数是不可变的。

In [ ]:
def memo(f):
        """Return a memoized version of single-argument function f."""
        cache = {}
        def memoized(n):
            if n not in cache:
                cache[n] = f(n)
            return cache[n]
        return memoized
fib = memo(fib)
fib(40)

由记忆函数节省的所需的计算时间总数在这个例子中是巨大的。被记忆的递归函数`fib`和迭代函数`fib_iter`都只需要线性于输入`n`的时间总数。为了计算`fib(40)`，`fib`的函数体只执行 40 次，而不是无记忆递归中的 102,334,155 次。

**空间。**为了理解函数所需的空间，我们必须在我们的计算模型中规定内存如何使用，保留和回收。在求解表达式过程中，我们必须保留所有活动环境和所有这些环境引用的值和帧。如果环境为表达式树当前分支中的一些表达式提供求值上下文，那么它就是活动环境。

例如，当求值`fib`时，解释器按序计算之前的每个值，遍历树形结构。为了这样做，它只需要在计算的任何时间点，跟踪树中在当前节点之前的那些节点。用于求出剩余节点的内存可以被回收，因为它不会影响未来的计算。通常，树形递归所需空间与树的深度成正比。

下面的图示描述了由求解`fib(3)`生成的表达式树。在求解`fib`最初调用的返回表达式的过程中，`fib(n-2)`被求值，产生值`0`。一旦这个值计算出来，对应的环境帧（标为灰色）就不再需要了：它并不是活动环境的一部分。所以，一个设计良好的解释器会回收用于储存这个帧的内存。另一方面，如果解释器当前正在求解`fib(n-1)`，那么由这次`fib`调用（其中`n`为`2`）创建的环境是活动的。与之对应，最开始在`3`上调用`fib`所创建的环境也是活动的，因为这个值还没有成功计算出来。

![](img/fib_env.png)

在`memo`的例子中，只要一些名称绑定到了活动环境中的某个函数上，关联到所返回函数（它包含`cache`）的环境必须保留。`cache`字典中的条目数量随传递给`fib`的唯一参数数量线性增长，它的规模线性于输入。另一方面，迭代实现只需要两个数值来在计算过程中跟踪：`prev`和`curr`，所以是常数大小。

我们使用记忆函数的例子展示了编程中的通用模式，即通常可以通过增加所用空间来减少计算时间，反之亦然。

## 3.2.4 示例：找零

考虑下面这个问题：如果给你半美元、四分之一美元、十美分、五美分和一美分，一美元有多少种找零的方式？更通常来说，我们能不能编写一个函数，使用一系列货币的面额，计算有多少种方式为给定的金额总数找零？

这个问题可以用递归函数简单解决。假设我们认为可用的硬币类型以某种顺序排列，假设从大到小排列。

使用`n`种硬币找零的方式为：

1.  使用所有除了第一种之外的硬币为`a`找零的方式，以及
2.  使用`n`种硬币为更小的金额`a - d`找零的方式，其中`d`是第一种硬币的面额。

为了弄清楚为什么这是正确的，可以看出，找零方式可以分为两组，不使用第一种硬币的方式，和使用它们的方式。所以，找零方式的总数等于不使用第一种硬币为该金额找零的方式数量，加上使用第一种硬币至少一次的方式数量。而后者的数量等于在使用第一种硬币之后，为剩余的金额找零的方式数量。

因此，我们可以递归将给定金额的找零问题，归约为使用更少种类的硬币为更小的金额找零的问题。仔细考虑这个归约原则，并且说服自己，如果我们规定了下列基本条件，我们就可以使用它来描述算法：

1.  如果`a`正好是零，那么有一种找零方式。
2.  如果`a`小于零，那么有零种找零方式。
3.  如果`n`小于零，那么有零种找零方式。

我们可以轻易将这个描述翻译成递归函数：

In [ ]:
def count_change(a, kinds=(50, 25, 10, 5, 1)):
        """Return the number of ways to change amount a using coin kinds."""
        if a == 0:
            return 1
        if a < 0 or len(kinds) == 0:
            return 0
        d = kinds[0]
        return count_change(a, kinds[1:]) + count_change(a - d, kinds)
count_change(100)

`count_change`函数生成树形递归过程，和`fib`的首个实现一样，它是重复的。它会花费很长时间来计算出`292`，除非我们记忆这个函数。另一方面，设计迭代算法来计算出结果的方式并不是那么明显，我们将它留做一个挑战。

## 3.2.5 增长度

前面的例子表明，不同过程在花费的时间和空间计算资源上有显著差异。我们用于描述这个差异的便捷方式，就是使用增长度的概念，来获得当输入变得更大时，过程所需资源的大致度量。

令`n`为度量问题规模的参数，`R(n)`为处理规模为`n`的问题的过程所需的资源总数。在我们前面的例子中，我们将`n`看做给定函数所要计算出的数值。但是还有其他可能。例如，如果我们的目标是计算某个数值的平方根近似值，我们会将`n`看做所需的有效位数的数量。通常，有一些问题相关的特性可用于分析给定的过程。与之相似，`R(n)`可用于度量所用的内存总数，所执行的基本的机器操作数量，以及其它。在一次只执行固定数量操作的计算中，用于求解表达式的所需时间，与求值过程中执行的基本机器操作数量成正比。

我们说，`R(n)`具有`Θ(f(n))`的增长度，写作`R(n)=Θ(f(n))`（读作“theta `f(n)`”），如果存在独立于`n`的常数`k1`和`k2`，那么对于任何足够大的`n`值：

```
k1·f(n) <= R(n) <= k2·f(n)
```

也就是说，对于较大的`n`，`R(n)`的值夹在两个具有`f(n)`规模的值之间：

+ 下界`k1·f(n)`，以及
+ 上界`k2·f(n)`。

例如，计算`n!`所需的步骤数量与`n`成正比，所以这个过程的所需步骤以`Θ(n)`增长。我们也看到了，递归实现`fact`的所需空间以`Θ(n)`增长。与之相反，迭代实现`fact_iter `花费相似的步骤数量，但是所需的空间保持不变。这里，我们说这个空间以`Θ(1)`增长。

我们的树形递归的斐波那契数计算函数`fib `的步骤数量，随输入`n`指数增长。尤其是，我们可以发现，第 n 个斐波那契数是距离`φ^(n-2)/√5`的最近整数，其中`φ`是黄金比例：

```
φ = (1 + √5)/2 ≈ 1.6180
```

我们也表示，步骤数量随返回值增长而增长，所以树形递归过程需要`Θ(φ^n)`的步骤，它的一个随`n`指数增长的函数。

增长度只提供了过程行为的大致描述。例如，需要`n^2`个步骤的过程和需要`1000·n^2`个步骤的过程，以及需要`3·n^2+10·n+17`个步骤的过程都拥有`Θ(n^2)`的增长度。在特定的情况下，增长度的分析过于粗略，不能在函数的两个可能实现中做出判断。

但是，增长度提供了实用的方法，来表示在改变问题规模的时候，我们应如何预期过程行为的改变。对于`Θ(n)`（线性）的过程，使规模加倍只会使所需的资源总数加倍。对于指数的过程，每一点问题规模的增长都会使所用资源以固定因数翻倍。接下来的例子展示了一个增长度为对数的算法，所以使问题规模加倍，只会使所需资源以固定总数增加。

## 3.2.6 示例：求幂


考虑对给定数值求幂的问题。我们希望有一个函数，它接受底数`b`和正整数指数`n`作为参数，并计算出`b^n`。一种方式就是通过递归定义：

```
b^n = b·b^(n-1)
b^0 = 1
```

这可以翻译成递归函数：

In [ ]:
def exp(b, n):
        if n == 0:
            return 1
        return b * exp(b, n-1)

这是个线性的递归过程，需要`Θ(n)`的步骤和空间。就像阶乘那样，我们可以编写等价的线性迭代形式，它需要相似的步骤数量，但只需要固定的空间。

In [ ]:
def exp_iter(b, n):
        result = 1
        for _ in range(n):
            result = result * b
        return result

我们可以以更少的步骤求幂，通过逐次平方。例如，我们这样计算`b^8`：

```
b·(b·(b·(b·(b·(b·(b·b))))))
```

我们可以使用三次乘法来计算它：

```
b^2 = b·b
b^4 = b^2·b^2
b^8 = b^4·b^4
```

这个方法对于 2 的幂的指数工作良好。我们也可以使用这个递归规则，在求幂中利用逐步平方的优点：

![](img/20160907175856.jpg)

我们同样可以将这个方式表达为递归函数：

In [ ]:
def square(x):
        return x*x
def fast_exp(b, n):
        if n == 0:
            return 1
        if n % 2 == 0:
            return square(fast_exp(b, n//2))
        else:
            return b * fast_exp(b, n-1)
fast_exp(2, 100)

`fast_exp`所生成的过程的空间和步骤数量随`n`以对数方式增长。为了弄清楚它，可以看出，使用`fast_exp`计算`b^2n`比计算`b^n`只需要一步额外的乘法操作。于是，我们能够计算的指数大小，在每次新的乘法操作时都会（近似）加倍。所以，计算`n`的指数所需乘法操作的数量，增长得像以`2`为底`n`的对数那样慢。这个过程拥有`Θ(log n)`的增长度。`Θ(log n)`和`Θ(n)`之间的差异在`n`非常大时变得显著。例如，`n`为`1000`时，`fast_exp `仅仅需要`14`个乘法操作，而不是`1000`。

# 3.3 递归数据结构

> 来源：[3.3   Recursive Data Structures](http://www-inst.eecs.berkeley.edu/~cs61a/sp12/book/interpretation.html#recursive-data-structures)

> 译者：[飞龙](https://github.com/wizardforcel)

> 协议：[CC BY-NC-SA 4.0](http://creativecommons.org/licenses/by-nc-sa/4.0/)

在第二章中，我们引入了偶对的概念，作为一种将两个对象结合为一个对象的机制。我们展示了偶对可以使用内建元素来实现。偶对的封闭性表明偶对的每个元素本身都可以为偶对。

这种封闭性允许我们实现递归列表的数据抽象，它是我们的第一种序列类型。递归列表可以使用递归函数最为自然地操作，就像它们的名称和结构表示的那样。在这一节中，我们会讨论操作递归列表和其它递归结构的自定义的函数。

## 3.3.1 处理递归列表

递归列表结构将列表表示为首个元素和列表的剩余部分的组合。我们之前使用函数实现了递归列表，但是现在我们可以使用类来重新实现。下面，长度（`__len__`）和元素选择（`__getitem__`）被重写来展示处理递归列表的典型模式。

In [ ]:
class Rlist(object):
        """A recursive list consisting of a first element and the rest."""
        class EmptyList(object):
            def __len__(self):
                return 0
        empty = EmptyList()
        def __init__(self, first, rest=empty):
            self.first = first
            self.rest = rest
        def __repr__(self):
            args = repr(self.first)
            if self.rest is not Rlist.empty:
                args += ', {0}'.format(repr(self.rest))
            return 'Rlist({0})'.format(args)
        def __len__(self):
            return 1 + len(self.rest)
        def __getitem__(self, i):
            if i == 0:
                return self.first
            return self.rest[i-1]

`__len__`和`__getitem__`的定义实际上是递归的，虽然不是那么明显。Python 内建函数`len`在自定义对象的参数上调用时会寻找叫做`__len__`的方法。与之类似，下标运算符会寻找叫做`__getitem__`的方法。于是，这些定义最后会调用对象自身。剩余部分上的递归调用是递归列表处理的普遍模式。这个递归列表的类定义与 Python 的内建序列和打印操作能够合理交互。

In [ ]:
s = Rlist(1, Rlist(2, Rlist(3)))
s.rest

In [ ]:
len(s)

In [ ]:
s[1]

创建新列表的操作能够直接使用递归来表示。例如，我们可以定义`extend_rlist`函数，它接受两个递归列表作为参数并将二者的元素组合到新列表中。

In [ ]:
def extend_rlist(s1, s2):
        if s1 is Rlist.empty:
            return s2
        return Rlist(s1.first, extend_rlist(s1.rest, s2))
extend_rlist(s.rest, s)

In [ ]:
Rlist(2, Rlist(3, Rlist(1, Rlist(2, Rlist(3)))))

与之类似，在递归列表上映射函数展示了相似的模式：

In [ ]:
def map_rlist(s, fn):
        if s is Rlist.empty:
            return s
        return Rlist(fn(s.first), map_rlist(s.rest, fn))
map_rlist(s, square)

In [ ]:
Rlist(1, Rlist(4, Rlist(9)))

过滤操作包括额外的条件语句，但是也拥有相似的递归结构。

In [ ]:
def filter_rlist(s, fn):
        if s is Rlist.empty:
            return s
        rest = filter_rlist(s.rest, fn)
        if fn(s.first):
            return Rlist(s.first, rest)
        return rest
filter_rlist(s, lambda x: x % 2 == 1)

In [ ]:
Rlist(1, Rlist(3))

列表操作的递归实现通常不需要局部赋值或者`while`语句。反之，递归列表可以作为函数调用的结果来拆分和构造。所以，它们拥有步骤数量和所需空间的线性增长度。

## 3.3.2 层次结构

层次结构产生于数据的封闭特性，例如，元组可以包含其它元组。考虑这个数值`1`到`4`的嵌套表示。

In [ ]:
((1, 2), 3, 4)

这个元组是个长度为 3 的序列，它的第一个元素也是一个元组。这个嵌套结构的盒子和指针的图示表明，它可以看做拥有四个叶子的树，每个叶子都是一个数值。

![](img/tree.png)

在树中，每个子树本身都是一棵树。作为基本条件，任何本身不是元组的元素都是一个简单的树，没有任何枝干。也就是说，所有数值都是树，就像在偶对`(1, 2)`和整个结构中那样。

递归是用于处理树形结构的自然工具，因为我们通常可以将树的操作降至枝干的操作，它会相应产生枝干的枝干的操作，以此类推，直到我们到达了树的叶子。例如，我们可以实现`count_leaves`函数，它返回树的叶子总数。

In [ ]:
t = ((1, 2), 3, 4)
count_leaves(t)

In [ ]:
big_tree = ((t, t), 5)
big_tree

In [ ]:
count_leaves(big_tree)

正如`map`是用于处理序列的强大工具，映射和递归一起为树的操作提供了强大而通用的计算形式。例如，我们可以使用高阶递归函数`map_tree `将树的每个叶子平方，它的结构类似于`count_leaves`。

In [ ]:
def map_tree(tree, fn):
        if type(tree) != tuple:
            return fn(tree)
        return tuple(map_tree(branch, fn) for branch in tree)

In [ ]:
map_tree(big_tree, square)

**内部值。**上面描述的树只在叶子上存在值。另一个通用的树形结构表示也在树的内部节点上存在值。我们使用类来表示这种树。

In [ ]:
class Tree(object):
        def __init__(self, entry, left=None, right=None):
            self.entry = entry
            self.left = left
            self.right = right
        def __repr__(self):
            args = repr(self.entry)
            if self.left or self.right:
                args += ', {0}, {1}'.format(repr(self.left), repr(self.right))
            return 'Tree({0})'.format(args)

例如，`Tree`类可以为`fib`的递归实现表示表达式树中计算的值。`fib`函数用于计算斐波那契数。下面的函数`fib_tree(n)`返回`Tree`，它将第 n 个斐波那契树作为`entry`，并将所有之前计算出来的斐波那契数存入它的枝干中。

In [ ]:
def fib_tree(n):
        """Return a Tree that represents a recursive Fibonacci calculation."""
        if n == 1:
            return Tree(0)
        if n == 2:
            return Tree(1)
        left = fib_tree(n-2)
        right = fib_tree(n-1)
        return Tree(left.entry + right.entry, left, right)
fib_tree(5)

这个例子表明，表达式树可以使用树形结构编程表示。嵌套表达式和树形数据结构的联系，在我们这一章稍后对解释器设计的讨论中起到核心作用。

## 3.3.3 集合

除了列表、元组和字典之外，Python 拥有第四种容器类型，叫做`set`。集合字面值遵循元素以花括号闭合的数学表示。重复的元素在构造中会移除。集合是无序容器，所以打印出来的顺序可能和元素在集合字面值中的顺序不同。

In [ ]:
s = {3, 2, 1, 4, 4}

In [ ]:
s

Python 的集合支持多种操作，包括成员测试、长度计算和标准的交集并集操作。

In [ ]:
3 in s

In [ ]:
len(s)

In [ ]:
s.union({1, 5})

In [ ]:
s.intersection({6, 5, 4, 3})

除了`union`和`intersection`，Python 的集合还支持多种其它操作。断言`isdisjoint`、`issubset`和`issuperset`提供了集合比较操作。集合是可变的，并且可以使用`add`、`remove`、`discard`和`pop`一次修改一个元素。额外的方法提供了多元素的修改，例如`clear`和`update`。Python [集合文档](http://docs.python.org/py3k/library/stdtypes.html#set)十分详细并足够易懂。

**实现集合。**抽象上，集合是不同对象的容器，支持成员测试、交集、并集和附加操作。向集合添加元素会返回新的集合，它包含原始集合的所有元素，如果没有重复的话，也包含新的元素。并集和交集运算返回出现在任意一个或两个集合中的元素构成的集合。就像任何数据抽象那样，我们可以随意实现任何集合表示上的任何函数，只要它们提供这种行为。

在这章的剩余部分，我们会考虑三个实现集合的不同方式，它们在表示上不同。我们会通过分析集合操作的增长度，刻画这些不同表示的效率。我们也会使用这一章之前的`Rlist`和`Tree`类，它们可以编写用于集合元素操作的简单而优雅的递归解决方案。

**作为无序序列的集合。**一种集合的表示方式是看做没有出现多于一次的元素的序列。空集由空序列来表示。成员测试会递归遍历整个列表。

In [ ]:
def empty(s):
        return s is Rlist.empty
def set_contains(s, v):
        """Return True if and only if set s contains v."""
        if empty(s):
            return False
        elif s.first == v:
            return True
        return set_contains(s.rest, v)
s = Rlist(1, Rlist(2, Rlist(3)))
set_contains(s, 2)

In [ ]:
set_contains(s, 5)

这个`set_contains `的实现需要`Θ(n)`的时间来测试元素的成员性，其中`n`是集合`s`的大小。使用这个线性时间的成员测试函数，我们可以将元素添加到集合中，也是线性时间。

In [ ]:
def adjoin_set(s, v):
        """Return a set containing all elements of s and element v."""
        if set_contains(s, v):
            return s
        return Rlist(v, s)
t = adjoin_set(s, 4)
t

那么问题来了，我们应该在设计表示时关注效率。计算两个集合`set1`和`set2`的交集需要成员测试，但是这次每个`set1`的元素必须测试`set2`中的成员性，对于两个大小为`n`的集合，这会产生步骤数量的平方增长度`Θ(n^2)`。

In [ ]:
def intersect_set(set1, set2):
        """Return a set containing all elements common to set1 and set2."""
        return filter_rlist(set1, lambda v: set_contains(set2, v))
intersect_set(t, map_rlist(s, square))

在计算两个集合的并集时，我们必须小心避免两次包含任意一个元素。`union_set `函数也需要线性数量的成员测试，同样会产生包含`Θ(n^2)`步骤的过程。

In [ ]:
def union_set(set1, set2):
        """Return a set containing all elements either in set1 or set2."""
        set1_not_set2 = filter_rlist(set1, lambda v: not set_contains(set2, v))
        return extend_rlist(set1_not_set2, set2)
union_set(t, s)

**作为有序元组的集合。**一种加速我们的集合操作的方式是修改表示，使集合元素递增排列。为了这样做，我们需要一些比较两个对象的方式，使我们能判断哪个更大。Python 中，许多不同对象类型都可以使用`<`和`>`运算符比较，但是我们会专注于这个例子中的数值。我们会通过将元素递增排列来表示数值集合。

有序的一个优点会在`set_contains`体现：在检查对象是否存在时，我们不再需要扫描整个集合。如果我们到达了大于要寻找的元素的集合元素，我们就知道这个元素不在集合中：

In [ ]:
def set_contains(s, v):
        if empty(s) or s.first > v:
            return False
        elif s.first == v:
            return True
        return set_contains(s.rest, v)
set_contains(s, 0)

这节省了多少步呢？最坏的情况中，我们所寻找的元素可能是集合中最大的元素，所以步骤数量和无序表示相同。另一方面，如果我们寻找许多不同大小的元素，我们可以预料到有时我们可以在列表开头的位置停止搜索，其它情况下我们仍旧需要检测整个列表。平均上我们应该需要检测集合中一半的元素。所以，步骤数量的平均值应该是`n/2`。这还是`Θ(n)`的增长度，但是它确实会在平均上为我们节省之前实现的一半步骤数量。

我们可以通过重新实现`intersect_set`获取更加可观的速度提升。在无序表示中，这个操作需要`Θ(n^2)`的步骤，因为我们对`set1`的每个元素执行`set2`上的完整扫描。但是使用有序的实现，我们可以使用更加机智的方式。我们同时迭代两个集合，跟踪`set1`中的元素`e1`和`set2`中的元素`e2`。当`e1`和`e2`相等时，我们在交集中添加该元素。

但是，假设`e1`小于`e2`，由于`e2`比`set2`的剩余元素更小，我们可以立即推断出`e1`不会出现在`set2`剩余部分的任何位置，因此也不会出现在交集中。所以，我们不再需要考虑`e1`，我们将它丢弃并来到`set1`的下一个元素。当`e2 < e1`时，我们可以使用相似的逻辑来步进`set2`中的元素。下面是这个函数：

In [ ]:
def intersect_set(set1, set2):
        if empty(set1) or empty(set2):
            return Rlist.empty
        e1, e2 = set1.first, set2.first
        if e1 == e2:
            return Rlist(e1, intersect_set(set1.rest, set2.rest))
        elif e1 < e2:
            return intersect_set(set1.rest, set2)
        elif e2 < e1:
            return intersect_set(set1, set2.rest)
intersect_set(s, s.rest)

为了估计这个过程所需的步骤数量，观察每一步我们都缩小了至少集合的一个元素的大小。所以，所需的步骤数量最多为`set1`和`set2`的大小之和，而不是无序表示所需的大小之积。这是`Θ(n)`而不是`Θ(n^2)`的增长度 -- 即使集合大小适中，它也是一个相当可观的加速。例如，两个大小为`100`的集合的交集需要 `200`步，而不是无序表示的 10000 步。

表示为有序序列的集合的添加和并集操作也以线性时间计算。这些实现都留做练习。

**作为二叉树的集合。**我们可以比有序列表表示做得更好，通过将几个元素重新以树的形式排列。我们使用之前引入的`Tree`类。树根的`entry`持有集合的一个元素。`left`分支的元素包括所有小于树根元素的元素。`right`分支的元素包括所有大于树根元素的元素。下面的图展示了一些树，它们表示集合`{1, 3, 5, 7, 9, 11}`。相同的集合可能会以不同形式的树来表示。有效表示所需的唯一条件就是所有`left`子树的元素应该小于`entry`，并且所有`right`子树的元素应该大于它。

![](img/set_trees.png)

树形表示的优点是：假设我们打算检查`v`是否在集合中。我们通过将`v`于`entry`比较开始。如果`v`小于它，我们就知道了我们只需要搜索`left`子树。如果`v`大于它，我们只需要搜索`right`子树。现在如果树是“平衡”的，每个这些子树都约为整个的一半大小。所以，每一步中我们都可以将大小为`n`的树的搜索问题降至搜索大小为`n/2`的子树。由于树的大小在每一步减半，我们应该预料到，用户搜索树的步骤以`Θ(log n)`增长。比起之前的表示，它的速度对于大型集合有可观的提升。`set_contains `函数利用了树形集合的有序结构：

In [ ]:
def set_contains(s, v):
        if s is None:
            return False
        elif s.entry == v:
            return True
        elif s.entry < v:
            return set_contains(s.right, v)
        elif s.entry > v:
            return set_contains(s.left, v)

向集合添加元素与之类似，并且也需要`Θ(log n)`的增长度。为了添加值`v`，我们将`v`与`entry`比较，来决定`v`应该添加到`right`还是`left`分支，以及是否已经将`v`添加到了合适的分支上。我们将这个新构造的分支与原始的`entry`和其它分支组合。如果`v`等于`entry`，我们就可以返回这个节点。如果我们被要求将`v`添加到空的树中，我们会生成一个`Tree`，它包含`v`作为`entry`，并且`left`和`right`都是空的分支。下面是这个函数：

In [ ]:
def adjoin_set(s, v):
        if s is None:
            return Tree(v)
        if s.entry == v:
            return s
        if s.entry < v:
            return Tree(s.entry, s.left, adjoin_set(s.right, v))
        if s.entry > v:
            return Tree(s.entry, adjoin_set(s.left, v), s.right)

adjoin_set(adjoin_set(adjoin_set(None, 2), 3), 1)

搜索该树可以以对数步骤数量执行，我们这个叙述基于树是“平衡”的假设。也就是说，树的左子树和右子树都拥有相同数量的相应元素，使每个子树含有母树一半的元素。但是我们如何确定，我们构造的树就是平衡的呢？即使我们以一颗平衡树开始，使用`adjoin_set`添加元素也会产生不平衡的结果。由于新添加的元素位置取决于如何将元素与集合中的已有元素比较，我们可以预测，如果我们“随机”添加元素到树中，树在平均上就会趋向于平衡。

但是这不是一个保证。例如，如果我们以空集开始，并向序列中添加 1 到 7，我们就会在最后得到很不平衡的树，其中所有左子树都是空的，所以它与简单的有序列表相比并没有什么优势。一种解决这个问题的方式是定义一种操作，它将任意的树转换为具有相同元素的平衡树。我们可以在每个`adjoin_set`操作之后执行这个转换来保证我们的集合是平衡的。

交集和并集操作可以在树形集合上以线性时间执行，通过将它们转换为有序的列表，并转换回来。细节留做练习。

**Python 集合实现。**Python 内建的`set`类型并没有使用上述任意一种表示。反之，Python 使用了一种实现，它的成员测试和添加操作是（近似）常量时间的，基于一种叫做哈希（散列）的机制，这是其它课程的话题。内建的 Python 集合不能包含可变的数据类型，例如列表、字典或者其它集合。为了能够嵌套集合，Python 也提供了一种内建的不可变`frozenset `类，除了可变操作和运算符之外，它拥有和`set`相同的方法。

# 3.4 异常

> 来源：[3.4   Exceptions](http://www-inst.eecs.berkeley.edu/~cs61a/sp12/book/interpretation.html#exceptions)

> 译者：[飞龙](https://github.com/wizardforcel)

> 协议：[CC BY-NC-SA 4.0](http://creativecommons.org/licenses/by-nc-sa/4.0/)

程序员必须总是留意程序中可能出现的错误。例子数不胜数：一个函数可能不会收到它预期的信息，必需的资源可能会丢失，或者网络上的连接可能丢失。在设计系统时，程序员必须预料到可能产生的异常情况并且采取适当地措施来处理它们。

处理程序中的错误没有单一的正确方式。为提供一些持久性服务而设计的程序，例如 Web 服务器 应该对错误健壮，将它们记录到日志中为之后考虑，而且在尽可能长的时间内继续接受新的请求。另一方面，Python 解释器通过立即终止以及打印错误信息来处理错误，便于程序员在错误发生时处理它。在任何情况下，程序员必须决定程序如何对异常条件做出反应。

异常是这一节的话题，它为程序的错误处理提供了通用的机制。产生异常是一种技巧，终止程序正常执行流，发射异常情况产生的信号，并直接返回到用于响应异常情况的程序的封闭部分。Python 解释器每次在检测到语句或表达式错误时抛出异常。用户也可以使用`raise`或`assert`语句来抛出异常。

**抛出异常。**异常是一个对象实例，它的类直接或间接继承自`BaseException`类。第一章引入的`assert`语句产生`AssertionError`类的异常。通常，异常实例可以使用`raise`语句来抛出。`raise`语句的通用形式在 [Python 文档](http://docs.python.org/py3k/reference/simple_stmts.html#raise)中描述。`raise`的最常见的作用是构造异常实例并抛出它。

In [ ]:
raise Exception('An error occurred')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
Exception: an error occurred

当异常产生时，当前代码块的语句不会继续执行。除非异常被解决了（下面会描述），解释器会直接返回到“读取-求值-打印”交互式循环中，或者在 Python 以文件参数启动的情况下会完全终止。此外，解释器会打印栈回溯，它是结构化的文本块，描述了执行分支中的一系列嵌套的活动函数，它们是异常产生的位置。在上面的例子中，文件名称`<stdin>`表示异常由用户在交互式会话中产生，而不是文件中的代码。

**处理异常。**异常可以使用封闭的`try`语句来处理。`try`语句由多个子句组成，第一个子句以`try`开始，剩下的以`except`开始。

In [ ]:
try:
    <try suite>
except <exception class> as <name>:
    <except suite>
...

当`try`语句执行时，`<try suite>`总是会立即执行。`except`子句组只在`<try suite>`执行过程中的异常产生时执行。每个`except`子句指定了需要处理的异常的特定类。例如，如果`<exception class>`是`AssertionError`，那么任何继承自`AssertionError`的类实例都会被处理，标识符`<name> `绑定到所产生的异常对象上，但是这个绑定在`<except suite>`之外并不有效。

例如，我们可以使用`try`语句来处理异常，在异常发生时将`x`绑定为`0`。

In [ ]:
try:
        x = 1/0
    except ZeroDivisionError as e:
        print('handling a', type(e))
        x = 0
handling a <class 'ZeroDivisionError'>

In [ ]:
x

`try`语句能够处理产生在函数体中的异常，函数在`<try suite>`中调用。当异常产生时，控制流会直接跳到最近的`try`语句的能够处理该异常类型的`<except suite>`的主体中。

In [ ]:
def invert(x):
        result = 1/x  # Raises a ZeroDivisionError if x is 0
        print('Never printed if x is 0')
        return result
def invert_safe(x):
        try:
            return invert(x)
        except ZeroDivisionError as e:
            return str(e)

In [ ]:
invert_safe(2)
Never printed if x is 0
0.5

In [ ]:
invert_safe(0)

这个例子表明，`invert`中的`print`表达式永远不会求值，反之，控制流跳到了`handler`中的`except`子句组中。将`ZeroDivisionError e`强制转为字符串会得到由`handler: 'division by zero'`返回的人类可读的字符串。

## 3.4.1 异常对象

异常对象本身就带有属性，例如在`assert`语句中的错误信息，以及有关异常产生处的信息。用户定义的异常类可以携带额外的属性。

在第一章中，我们实现了牛顿法来寻找任何函数的零点。下面的例子定义了一个异常类，无论何时`ValueError`出现，它都返回迭代改进过程中所发现的最佳猜测值。数学错误（`ValueError`的一种）在`sqrt`在负数上调用时产生。这个异常由抛出`IterImproveError`处理，它将牛顿迭代法的最新猜测值储存为参数。

首先，我们定义了新的类，继承自`Exception`。

In [ ]:
class IterImproveError(Exception):
        def __init__(self, last_guess):
            self.last_guess = last_guess

下面，我们定义了`IterImprove`，我们的通用迭代改进算法的一个版本。这个版本通过抛出`IterImproveError`异常，储存最新的猜测值来处理任何`ValueError`。像之前一样，`iter_improve`接受两个函数作为参数，每个函数都接受单一的数值参数。`update`函数返回新的猜测值，而`done`函数返回布尔值，表明改进是否收敛到了正确的值。

In [ ]:
def iter_improve(update, done, guess=1, max_updates=1000):
        k = 0
        try:
            while not done(guess) and k < max_updates:
                guess = update(guess)
                k = k + 1
            return guess
        except ValueError:
            raise IterImproveError(guess)

最后，我们定义了`find_root`，它返回`iter_improve`的结果。`iter_improve`应用于由`newton_update`返回的牛顿更新函数。`newton_update`定义在第一章，在这个例子中无需任何改变。`find_root`的这个版本通过返回它的最后一个猜测之来处理`IterImproveError`。

In [ ]:
def find_root(f, guess=1):
        def done(x):
            return f(x) == 0
        try:
            return iter_improve(newton_update(f), done, guess)
        except IterImproveError as e:
            return e.last_guess

考虑使用`find_root`来寻找`2 * x ** 2 + sqrt(x)`的零点。这个函数的一个零点是`0`，但是在任何负数上求解它会产生`ValueError`。我们第一章的牛顿法实现会产生异常，并且不能返回任何零点的猜测值。我们的修订版实现在错误之前返回了最新的猜测值。

In [ ]:
from math import sqrt
find_root(lambda x: 2*x*x + sqrt(x))

虽然这个近似值仍旧距离正确的答案`0`很远，一些应用更倾向于这个近似值而不是`ValueError`。

异常是另一个技巧，帮助我们将程序细节划分为模块化的部分。在这个例子中，Python 的异常机制允许我们分离迭代改进的逻辑，它在`try`子句组中没有发生改变，以及错误处理的逻辑，它出现在`except`子句中。我们也会发现，异常在使用 Python 实现解释器时是个非常实用的特性。

# 3.5 组合语言的解释器

> 来源：[3.5   Interpreters for Languages with Combination](http://www-inst.eecs.berkeley.edu/~cs61a/sp12/book/interpretation.html#interpreters-for-languages-with-combination)

> 译者：[飞龙](https://github.com/wizardforcel)

> 协议：[CC BY-NC-SA 4.0](http://creativecommons.org/licenses/by-nc-sa/4.0/)

运行在任何现代计算机上的软件都以多种编程语言写成。其中有物理语言，例如用于特定计算机的机器语言。这些语言涉及到基于独立储存位和原始机器指令的数据表示和控制。机器语言的程序员涉及到使用提供的硬件，为资源有限的计算构建系统和功能的高效实现。高阶语言构建在机器语言之上，隐藏了表示为位集的数据，以及表示为原始指令序列的程序的细节。这些语言拥有例如过程定义的组合和抽象的手段，它们适用于组织大规模的软件系统。

元语言抽象 -- 建立了新的语言 -- 并在所有工程设计分支中起到重要作用。它对于计算机编程尤其重要，因为我们不仅仅可以在编程中构想出新的语言，我们也能够通过构建解释器来实现它们。编程语言的解释器是一个函数，它在语言的表达式上调用，执行求解表达式所需的操作。

我们现在已经开始了技术之旅，通过这种技术，编程语言可以建立在其它语言之上。我们首先会为计算器定义解释器，它是一种受限的语言，和 Python 调用表达式具有相同的语法。我们之后会从零开始开发 Scheme 和 Logo 语言的解释器，它们都是 Lisp 的方言，Lisp 是现在仍旧广泛使用的第二老的语言。我们所创建的解释器，在某种意义上，会让我们使用 Logo 编写完全通用的程序。为了这样做，它会实现我们已经在这门课中开发的求值环境模型。

## 3.5.1 计算器

我们的第一种新语言叫做计算器，一种用于加减乘除的算术运算的表达式语言。计算器拥有 Python 调用表达式的语法，但是它的运算符对于所接受的参数数量更加灵活。例如，计算器运算符`mul`和`add`可接受任何数量的参数：

In [ ]:
calc> add(1, 2, 3, 4)

In [ ]:
calc> mul()

`sub`运算符拥有两种行为：传入一个运算符，它会对运算符取反。传入至少两个，它会从第一个参数中减掉剩余的参数。`div`运算符拥有 Python 的`operator.truediv`的语义，只接受两个参数。

In [ ]:
calc> sub(10, 1, 2, 3)

In [ ]:
calc> sub(3)

In [ ]:
calc> div(15, 12)

就像 Python 中那样，调用表达式的嵌套提供了计算器语言中的组合手段。为了精简符号，我们使用运算符的标准符号来代替名称：

In [ ]:
calc> sub(100, mul(7, add(8, div(-12, -3))))

In [ ]:
calc> -(100, *(7, +(8, /(-12, -3))))

我们会使用 Python 实现计算器解释器。也就是说，我们会编写 Python 程序来接受字符串作为输入，并返回求值结果。如果输入是符合要求的计算器表达式，结果为字符串，反之会产生合适的异常。计算器语言解释器的核心是叫做`calc_eval`的递归函数，它会求解树形表达式对象。

**表达式树。**到目前为止，我们在描述求值过程中所引用的表达式树，还是概念上的实体。我们从没有显式将表达式树表示为程序中的数据。为了编写解释器，我们必须将表达式当做数据操作。在这一章中，许多我们之前介绍过的概念都会最终以代码实现。

计算器中的基本表达式只是一个数值，类型为`int`或`float`。所有复合表达式都是调用表达式。调用表达式表示为拥有两个属性实例的`Exp`类。计算器的`operator`总是字符串：算数运算符的名称或符号。`operands`要么是基本表达式，要么是`Exp`的实例本身。

In [ ]:
 class Exp(object):
        """A call expression in Calculator."""
        def __init__(self, operator, operands):
            self.operator = operator
            self.operands = operands
        def __repr__(self):
            return 'Exp({0}, {1})'.format(repr(self.operator), repr(self.operands))
        def __str__(self):
            operand_strs = ', '.join(map(str, self.operands))
            return '{0}({1})'.format(self.operator, operand_strs)

`Exp`实例定义了两个字符串方法。`__repr__`方法返回 Python 表达式，而`__str__`方法返回计算器表达式。

In [ ]:
Exp('add', [1, 2])

In [ ]:
str(Exp('add', [1, 2]))

In [ ]:
Exp('add', [1, Exp('mul', [2, 3, 4])])

In [ ]:
str(Exp('add', [1, Exp('mul', [2, 3, 4])]))

最后的例子演示了`Exp`类如何通过包含作为`operands`元素的`Exp`的实例，来表示表达式树中的层次结构。

**求值。**`calc_eval`函数接受表达式作为参数，并返回它的值。它根据表达式的形式为表达式分类，并且指导它的求值。对于计算器来说，表达式的两种句法形式是数值或调用表达式，后者是`Exp`的实例。数值是自求值的，它们可以直接从`calc_eval`中返回。调用表达式需要使用函数。

调用表达式首先通过将`calc_eval`函数递归映射到操作数的列表，计算出参数列表来求值。之后，在第二个函数`calc_apply`中，运算符会作用于这些参数上。

计算器语言足够简单，我们可以轻易地在单一函数中表达每个运算符的使用逻辑。在`calc_apply`中，每种条件子句对应一个运算符。

In [ ]:
from operator import mul
from functools import reduce
def calc_apply(operator, args):
        """Apply the named operator to a list of args."""
        if operator in ('add', '+'):
            return sum(args)
        if operator in ('sub', '-'):
            if len(args) == 0:
                raise TypeError(operator + ' requires at least 1 argument')
            if len(args) == 1:
                return -args[0]
            return sum(args[:1] + [-arg for arg in args[1:]])
        if operator in ('mul', '*'):
            return reduce(mul, args, 1)
        if operator in ('div', '/'):
            if len(args) != 2:
                raise TypeError(operator + ' requires exactly 2 arguments')
            numer, denom = args
            return numer/denom

上面，每个语句组计算了不同运算符的结果，或者当参数错误时产生合适的`TypeError`。`calc_apply`函数可以直接调用，但是必须传入值的列表作为参数，而不是运算符表达式的列表。

In [ ]:
calc_apply('+', [1, 2, 3])

In [ ]:
calc_apply('-', [10, 1, 2, 3])

In [ ]:
calc_apply('*', [])

In [ ]:
calc_apply('/', [40, 5])

`calc_eval`的作用是，执行合适的`calc_apply`调用，通过首先计算操作数子表达式的值，之后将它们作为参数传入`calc_apply`。于是，`calc_eval`可以接受嵌套表达式。

In [ ]:
e = Exp('add', [2, Exp('mul', [4, 6])])

In [ ]:
str(e)

In [ ]:
calc_eval(e)

`calc_eval`的结构是个类型（表达式的形式）分发的例子。第一种表达式是数值，不需要任何的额外求值步骤。通常，基本表达式不需要任何额外的求值步骤，这叫做自求值。计算器语言中唯一的自求值表达式就是数值，但是在通用语言中可能也包括字符串、布尔值，以及其它。

**“读取-求值-打印”循环。**和解释器交互的典型方式是“读取-求值-打印”循环（REPL），它是一种交互模式，读取表达式、对其求值，之后为用户打印出结果。Python 交互式会话就是这种循环的例子。

REPL 的实现与所使用的解释器无关。下面的`read_eval_print_loop`函数使用内建的`input`函数，从用户接受一行文本作为输入。它使用语言特定的`calc_parse`函数构建表达式树。`calc_parse`在随后的解析一节中定义。最后，它打印出对由`calc_parse`返回的表达式树调用`calc_eval`的结果。

In [ ]:
def read_eval_print_loop():
        """Run a read-eval-print loop for calculator."""
        while True:
            expression_tree = calc_parse(input('calc> '))
            print(calc_eval(expression_tree))

`read_eval_print_loop`的这个版本包含所有交互式界面的必要组件。一个样例会话可能像这样：

In [ ]:
calc> mul(1, 2, 3)

In [ ]:
calc> add()

In [ ]:
calc> add(2, div(4, 8))

这个循环没有实现终端或者错误处理机制。我们可以通过向用户报告错误来改进这个界面。我们也可以允许用户通过发射键盘中断信号（`Control-C`），或文件末尾信号（`Control-D`）来退出循环。为了实现这些改进，我们将原始的`while`语句组放在`try`语句中。第一个`except`子句处理了由`calc_parse`产生的`SyntaxError`异常，也处理了由`calc_eval`产生的`TypeError`和`ZeroDivisionError`异常。

In [ ]:
def read_eval_print_loop():
        """Run a read-eval-print loop for calculator."""
        while True:
            try:
                expression_tree = calc_parse(input('calc> '))
                print(calc_eval(expression_tree))
            except (SyntaxError, TypeError, ZeroDivisionError) as err:
                print(type(err).__name__ + ':', err)
            except (KeyboardInterrupt, EOFError):  # <Control>-D, etc.
                print('Calculation completed.')
                return

这个循环实现报告错误而不退出循环。发生错误时不退出程序，而是在错误消息之后重新开始循环可以让用户回顾他们的表达式。通过导入`readline`模块，用户甚至可以使用上箭头或`Control-P`来回忆他们之前的输入。最终的结果提供了错误信息报告的界面：

In [ ]:
calc> add
SyntaxError: expected ( after add

In [ ]:
calc> div(5)
TypeError: div requires exactly 2 arguments

In [ ]:
calc> div(1, 0)
ZeroDivisionError: division by zero

In [ ]:
calc> ^DCalculation completed.

在我们将解释器推广到计算器之外的语言时，我们会看到，`read_eval_print_loop`由解析函数、求值函数，和由`try`语句处理的异常类型参数化。除了这些修改之外，任何 REPL 都可以使用相同的结构来实现。

## 3.5.2 解析

解析是从原始文本输入生成表达式树的过程。解释这些表达式树是求值函数的任务，但是解析器必须提供符合格式的表达式树给求值器。解析器实际上由两个组件组成，词法分析器和语法分析器。首先，词法分析器将输入字符串拆成标记（token），它们是语言的最小语法单元，就像名称和符号那样。其次，语法分析器从这个标记序列中构建表达式树。

In [ ]:
def calc_parse(line):
        """Parse a line of calculator input and return an expression tree."""
        tokens = tokenize(line)
        expression_tree = analyze(tokens)
        if len(tokens) > 0:
            raise SyntaxError('Extra token(s): ' + ' '.join(tokens))
        return expression_tree

标记序列由叫做`tokenize`的词法分析器产生，并被叫做`analyze`语法分析器使用。这里，我们定义了`calc_parse`，它只接受符合格式的计算器表达式。一些语言的解析器为接受以换行符、分号或空格分隔的多种表达式而设计。我们在引入 Logo 语言之前会推迟实现这种复杂性。

**词法分析。**用于将字符串解释为标记序列的组件叫做分词器（tokenizer ），或者词法分析器。在我们的视线中，分词器是个叫做`tokenize`的函数。计算器语言由包含数值、运算符名称和运算符类型的符号（比如`+`）组成。这些符号总是由两种分隔符划分：逗号和圆括号。每个符号本身都是标记，就像每个逗号和圆括号那样。标记可以通过向输入字符串添加空格，之后在每个空格处分割字符串来分开。

In [ ]:
def tokenize(line):
        """Convert a string into a list of tokens."""
        spaced = line.replace('(',' ( ').replace(')',' ) ').replace(',', ' , ')
        return spaced.split()

对符合格式的计算器表达式分词不会损坏名称，但是会分开所有符号和分隔符。

In [ ]:
tokenize('add(2, mul(4, 6))')
['add', '(', '2', ',', 'mul', '(', '4', ',', '6', ')', ')']

拥有更加复合语法的语言可能需要更复杂的分词器。特别是，许多分析器会解析每种返回标记的语法类型。例如，计算机中的标记类型可能是运算符、名称、数值或分隔符。这个分类可以简化标记序列的解析。

**语法分析。**将标记序列解释为表达式树的组件叫做语法分析器。在我们的实现中，语法分析由叫做`analyze`的递归函数完成。它是递归的，因为分析标记序列经常涉及到分析这些表达式树中的标记子序列，它本身作为更大的表达式树的子分支（比如操作数）。递归会生成由求值器使用的层次结构。

`analyze`函数接受标记列表，以符合格式的表达式开始。它会分析第一个标记，将表示数值的字符串强制转换为数字的值。之后要考虑计算机中的两个合法表达式类型。数字标记本身就是完整的基本表达式树。复合表达式以运算符开始，之后是操作数表达式的列表，由圆括号分隔。我们以一个不检查语法错误的实现开始。

In [ ]:
def analyze(tokens):
        """Create a tree of nested lists from a sequence of tokens."""
        token = analyze_token(tokens.pop(0))
        if type(token) in (int, float):
            return token
        else:
            tokens.pop(0)  # Remove (
            return Exp(token, analyze_operands(tokens))
def analyze_operands(tokens):
        """Read a list of comma-separated operands."""
        operands = []
        while tokens[0] != ')':
            if operands:
                tokens.pop(0)  # Remove ,
            operands.append(analyze(tokens))
        tokens.pop(0)  # Remove )
        return operands

最后，我们需要实现`analyze_token`。`analyze_token`函数将数值文本转换为数值。我们并不自己实现这个逻辑，而是依靠内建的 Python 类型转换，使用`int`和`float`构造器来将标记转换为这种类型。

In [ ]:
def analyze_token(token):
        """Return the value of token if it can be analyzed as a number, or token."""
        try:
            return int(token)
        except (TypeError, ValueError):
            try:
                return float(token)
            except (TypeError, ValueError):
                return token

我们的`analyze`实现就完成了。它能够正确将符合格式的计算器表达式解析为表达式树。这些树由`str`函数转换回计算器表达式。

In [ ]:
expression = 'add(2, mul(4, 6))'
analyze(tokenize(expression))

In [ ]:
str(analyze(tokenize(expression)))

`analyse`函数只会返回符合格式的表达式树，并且它必须检测输入中的语法错误。特别是，它必须检测表达式是否完整、正确分隔，以及只含有已知的运算符。下面的修订版本确保了语法分析的每一步都找到了预期的标记。

In [ ]:
known_operators = ['add', 'sub', 'mul', 'div', '+', '-', '*', '/']
def analyze(tokens):
        """Create a tree of nested lists from a sequence of tokens."""
        assert_non_empty(tokens)
        token = analyze_token(tokens.pop(0))
        if type(token) in (int, float):
            return token
        if token in known_operators:
            if len(tokens) == 0 or tokens.pop(0) != '(':
                raise SyntaxError('expected ( after ' + token)
            return Exp(token, analyze_operands(tokens))
        else:
            raise SyntaxError('unexpected ' + token)
def analyze_operands(tokens):
        """Analyze a sequence of comma-separated operands."""
        assert_non_empty(tokens)
        operands = []
        while tokens[0] != ')':
            if operands and tokens.pop(0) != ',':
                raise SyntaxError('expected ,')
            operands.append(analyze(tokens))
            assert_non_empty(tokens)
        tokens.pop(0)  # Remove )
        return elements
def assert_non_empty(tokens):
        """Raise an exception if tokens is empty."""
        if len(tokens) == 0:
            raise SyntaxError('unexpected end of line')

大量的语法错误在本质上提升了解释器的可用性。在上面，`SyntaxError `异常包含所发生的问题描述。这些错误字符串也用作这些分析函数的定义文档。

这个定义完成了我们的计算器解释器。你可以获取单独的 Python 3 源码 [`calc.py`](http://www-inst.eecs.berkeley.edu/~cs61a/sp12/book/calc.py)来测试。我们的解释器对错误健壮，用户在`calc>`提示符后面的每个输入都会求值为数值，或者产生合适的错误，描述输入为什么不是符合格式的计算器表达式。

# 3.6 抽象语言的解释器

> 来源：[3.6   Interpreters for Languages with Abstraction](http://www-inst.eecs.berkeley.edu/~cs61a/sp12/book/interpretation.html#interpreters-for-languages-with-abstraction)

> 译者：[飞龙](https://github.com/wizardforcel)

> 协议：[CC BY-NC-SA 4.0](http://creativecommons.org/licenses/by-nc-sa/4.0/)

计算器语言提供了一种手段，来组合一些嵌套的调用表达式。然而，我们却没有办法定义新的运算符，将值赋给名称，或者表达通用的计算方法。总之，计算器并不以任何方式支持抽象。所以，它并不是特别强大或通用的编程语言。我们现在转到定义一种通用编程语言的任务中，这门语言通过将名称绑定到值以及定义新的操作来支持抽象。

我们并不是进一步扩展简单的计算器语言，而是重新开始，并且为 Logo 语言开发解释器。Logo 并不是为这门课发明的语言，而是一种经典的命令式语言，拥有许多解释器实现和自己的开发者社区。

上一章，我们将完整的解释器表示为 Python 源码，这一章使用描述性的方式。某个配套工程需要你通过构建完整的 Logo 函数式解释器来实现这里展示的概念。

## 3.6.1 Scheme 语言

Scheme 是 Lisp 的一种方言，Lisp 是现在仍在广泛使用的第二老（在 Fortran 之后）的编程语言。Scheme首次在 1975 年由 Gerald Sussman 和 Guy Steele 描述。Revised(4) Report on the Algorithmic Language Scheme 的引言中写道：

> 编程语言不应该通过堆砌特性，而是应该通过移除那些使额外特性变得必要的缺点和限制来设计。Scheme 表明，用于组成表达式的非常少量的规则，在没有组合方式的限制的情况下，足以组成实用并且高效的编程语言，它足够灵活，在使用中可以支持多数当今的主流编程范式。

我们将这个报告推荐给你作为 Scheme 语言的详细参考。我们这里只会涉及重点。下面的描述中，我们会用到报告中的例子。

虽然 Scheme 非常简单，但它是一种真正的编程语言，在许多地方都类似于 Python，但是“语法糖[1]”会尽量少。基本上，所有运算符都是函数调用的形式。这里我们会描述完整的 Scheme 语言的在报告中描述的可展示的子集。

> [1] 非常遗憾，这对于 Scheme 语言的最新版本并不成立，就像 Revised(6) Report 中的那样。所以这里我们仅仅针对之前的版本。

Scheme 有多种可用的实现，它们添加了额外的过程。在 UCB，我们使用[Stk 解释器的一个修改版](http://inst.eecs.berkeley.edu/~scheme/)，它也在我们的教学服务器上以`stk`提供。不幸的是，它并不严格遵守正式规范，但它可用于我们的目的。

**使用解释器。**就像 Python 解释器[2]那样，向 Stk 键入的表达式会由“读取-求值-打印”循环求值并打印：

In [ ]:
3

In [ ]:
(- (/ (* (+ 3 7 10) (- 1000 8)) 992) 17)

In [ ]:
(define (fib n) (if (< n 2) n (+ (fib (- n 2)) (fib (- n 1)))))

In [ ]:
'(1 (7 19))

> [2] 在我们的例子中，我们使用了和 Python 相同的符号`>>>`和`...`，来表示解释器的输入行，和非前缀输出的行。实际上，Scheme 解释器使用不同的提示符。例如，Stk 以`STk>`来提示，并且不提示连续行。然而 Python 的惯例使输入和输出更加清晰。

**Scheme 中的值。**Scheme 中的值通常与 Python 对应。

布尔值

真值和假值，使用`#t`和`#f`来表示。Scheme 中唯一的假值（按照 Python 的含义）就是`#f`。

数值

这包括任意精度的整数、有理数、复数，和“不精确”（通常是浮点）数值。整数可用标准的十进制表示，或者通过在数字之前添加`#o`（八进制）、`#x`（十六进制）或`#b`（二进制），以其他进制表示。

符号

符号是一种字符串，但是不被引号包围。有效的字符包括字母、数字和：

```
!  $  %  &  *  /  :  <  = >  ?  ^  _  ~  +  -  .  @
```

在使用`read`函数输入时，它会读取 Scheme 表达式（也是解释器用于输入程序文本的东西），不区分符号中的大小写（在STk 实现中会转为小写）。两个带有相同表示的符号表示同一对象（并不是两个碰巧拥有相同内容的对象）。

偶对和列表

偶对是含有两个（任意类型）成员的对象，叫做它的`car`和`cdr`。`car`为`A`且`cdr`为`B`的偶对可表示为`(A . B)`。偶对（就像 Python 中的元组）可以表示列表、树和任意的层次结构。

标准的 Scheme 列表包含空的列表值（记为`()`），或者包含一个偶对，它的`car`是列表第一个元素，`cdr`是列表的剩余部分。所以，包含整数`1, 2, 3`的列表可表示为：

```scheme
(1 . (2 . (3 . ())))
```

列表无处不在，Scheme 允许我们将`(a . ())`缩略为`(a)`，将`(a . (b ...))`缩略为`(a b ...)`。所以，上面的列表通常写为：

```scheme
(1 2 3)
```

过程（函数）

就像 Python 中一样，过程（或函数）值表示一些计算，它们可以通过向函数提供参数来调用。过程要么是原始的，由 Scheme 的运行时系统提供，要么从 Scheme 表达式和环境构造（就像 Python 中那样）。没有用于函数值的直接表示，但是有一些绑定到基本函数的预定义标识符，也有一些 Scheme 表达式，在求值时会产生新的过程值。

其它类型

Scheme 也支持字符和字符串（类似 Python 的字符串，除了 Scheme 区分字符和字符串），以及向量（就像 Python 的列表）。

**程序表示。**就像其它 Lisp 版本，Scheme 的数据值也用于表示程序。例如，下面的 Scheme 列表：

```scheme
(+ x (* 10 y))
```

取决于如何使用，可表示为三个元素的列表（它的最后一个元素也是三个元素的列表），或者表达为用于计算`x+10y`的 Scheme 表达式。为了将 Scheme 值求值为程序，我们需要考虑值的类型，并按以下步骤求值：

+ 整数、布尔值、字符、字符串和向量都求值为它们自己。所以，表达式`5`求值为 5。
+ 纯符号看做变量。它们的值由当前被求值环境来决定，就像 Python 那样。
+ 非空列表以两种方式解释，取决于它们的第一个成员：
  + 如果第一个成员是特殊形式的符号（在下面描述），求值由这个特殊形式的规则执行。
  + 所有其他情况（叫做组合）中，列表的成员会以非特定的顺序（递归）求值。第一个成员必须是函数值。这个值会被调用，以列表中剩余成员的值作为参数。
+ 其他 Scheme 值（特别是，不是列表的偶对）在程序中是错误的。

例如：

In [ ]:
5              ; A literal.

In [ ]:
(define x 3)   ; A special form that creates a binding for symbol

In [ ]:
(+ 3 (* 10 x)) ; A combination.  Symbol + is bound to the primitive
33                  ; add function and * to primitive multiply.

**基本的特殊形式。**特殊形式将东西表示为 Python 中的控制结构、函数调用或者类的定义：在调用时，这些结构不会简单地立即求值。

首先，一些通用的结构以这种形式使用：

`EXPR-SEQ`

只是表达式的序列，例如：

```scheme
(+ 3 2) x (* y z)
```

当它出现在下面的定义中时，它指代从左到右求值的表达式序列，序列中最后一个表达式的值就是它的值。

`BODY`

一些结构拥有“主体”，它们是 `EXPR-SEQ`，就像上面一样，可能由一个或多个定义处理。它们的值就是 `EXPR-SEQ` 的值。这些定义的解释请见内部定义一节。

下面是这些特殊形式的代表性子集：

定义

定义可以出现在程序的顶层（也就是不包含在其它结构中）。

`(define SYM EXPR)`

求出`EXPR`并在当前环境将其值绑定到符号`SYM`上。

`(define (SYM ARGUMENTS) BODY)`

等价于`(define SYM (lambda (ARGUMENTS) BODY))`。

`(lambda (ARGUMENTS) BODY)`

求值为函数。`ARGUMENTS `通常为（可能非空的）不同符号的列表，向函数提供参数名称，并且表明它们的数量。`ARGUMENTS`也可能具有如下形式：

```scheme
(sym1 sym2 ... symn . symr)
```

（也就是说，列表的末尾并不像普通列表那样是空的，最后的`cdr`是个符号。）这种情况下，`symr`会绑定到列表的尾后参数值（后面的第 n+1 个参数）。

当产生的函数被调用时，`ARGUMENTS`在一个新的环境中绑定到形参的值上，新的环境扩展自`lambda`表达式求值的环境（就像 Python 那样）。之后`BODY`会被求值，它的值会作为调用的值返回。

`(if COND-EXPR TRUE-EXPR OPTIONAL-FALSE-EXPR)`

求出`COND-EXPR`，如果它的值不是`#f`，那么求出`TRUE-EXPR`，结果会作为`if`的值。如果`COND-EXPR`值为`#f`而且`OPTIONAL-FALSE-EXPR`存在，它会被求值为并作为`if`的值。如果它不存在，`if`值是未定义的。

`(set! SYMBOL EXPR)`

求出`EXPR`使用该值替换`SYMBOL `的绑定。`SYMBOL `必须已经绑定，否则会出现错误。和 Python 的默认情况不同，它会在定义它的第一个环境帧上替换绑定，而不总是最深处的帧。

`(quote EXPR)` 或 `'EXPR`

将 Scheme 数据结构用于程序表示的一个问题，是需要一种方式来表示打算被求值的程序文本。`quote`形式求值为`EXPR`自身，而不进行进一步的求值（替代的形式使用前导的单引号，由 Scheme 表达式读取器转换为第一种形式）。例如：

In [ ]:
(+ 1 2)

In [ ]:
'(+ 1 2)

In [ ]:
(define x 3)

In [ ]:
(define x 3)

In [ ]:
x

In [ ]:
(quote x)

In [ ]:
'5

In [ ]:
(quote 'x)

**派生的特殊形式**

派生结构时可以翻译为基本结构的结构。它们的目的是让程序对于读取器更加简洁可读。在 Scheme 中：

`(begin EXPR-SEQ)`

简单地求值并产生`EXPR-SEQ`的值。这个结构是个简单的方式，用于在需要单个表达式的上下文中执行序列或表达式。

`(and EXPR1 EXPR2 ...)`

每个`EXPR`从左到右执行，直到碰到了`#f`，或遍历完`EXPRs`。值是最后求值的`EXPR`，如果`EXPRs`列表为空，那么值为`#t`。例如：

In [ ]:
(and (= 2 2) (> 2 1))

In [ ]:
(and (< 2 2) (> 2 1))

In [ ]:
(and (= 2 2) '(a b))

In [ ]:
(and)

`(or EXPR1 EXPR2 ...)`

每个`EXPR`从左到右求值，直到碰到了不为`#f`的值，或遍历完`EXPRs`。值为最后求值的`EXPR`，如`EXPRs`列表为空，那么值为`#f`。例如：

(or (= 2 2) (> 2 3))

(or (= 2 2) '(a b))

(or (> 2 2) '(a b))

(or (> 2 2) (> 2 3))

(or)

`(cond CLAUSE1 CLAUSE2 ...)`

每个`CLAUSEi`都依次处理，直到其中一个处理成功，它的值就是`cond`的值。如果没有子句处理成功，值是未定义的。每个子句都有三种可能的形式。

如果`TEST-EXPR `求值为不为`#f`的值，`(TEST-EXPR EXPR-SEQ)`形式执行成功。这种情况下，它会求出`EXPR-SEQ`并产生它的值。`EXPR-SEQ`可以不写，这种情况下值为`TEST-EXPR`本身。

最后一个子句可为`(else EXPR-SEQ)`的形式，它等价于`(#t EXPR-SEQ)`。

最后，如果`(TEST_EXPR => EXPR)`的形式在`TEST_EXPR`求值为不为`#f`的值（叫做`V`）时求值成功。如果求值成功，`cond`结构的值是由`(EXPR V)`返回的值。也就是说，`EXPR`必须求值为单参数的函数，在`TEST_EXPR`的值上调用。

例如：

In [ ]:
(cond ((> 3 2) 'greater)
...        ((< 3 2) 'less)))
greater

In [ ]:
(cond ((> 3 3) 'greater)
...        ((< 3 3) 'less)
...        (else 'equal))
equal

In [ ]:
(cond ((if (< -2 -3) #f -3) => abs)
...        (else #f))
3
```

`(case KEY-EXPR CLAUSE1 CLAUSE2 ...)`

`KEY-EXPR`的求值会产生一个值`K`。之后将`K`与每个`CLAUSEi`一次匹配，直到其中一个成功，并且返回该子句的值。如果没有子句成功，值是未定义的。每个子句都拥有`((DATUM1 DATUM2 ...) EXPR-SEQ)`的形式。其中`DATUMs`是 Scheme 值（它们不会被求值）。如果`K`匹配了`DATUM`的值之一（由下面描述的`eqv?`函数判断），子句就会求值成功，它的`EXPR-SEQ`就会被求值，并且它的值会作为`case`的值。最后的子句可为`(else EXPR-SEQ)`的形式，它总是会成功，例如：

In [ ]:
(case (* 2 3)
...     ((2 3 5 7) 'prime)
...     ((1 4 6 8 9) 'composite))

In [ ]:
(case (car '(a . b))
...     ((a c) 'd)
...     ((b 3) 'e))

In [ ]:
(case (car '(c d))
...    ((a e i o u) 'vowel)
...    ((w y) 'semivowel)
...    (else 'consonant))

`(let BINDINGS BODY)`

`BINDINGS`是偶对的列表，形式为：

```scheme
( (VAR1 INIT1) (VAR2 INIT2) ...)
```

其中`VARs`是（不同的）符号，而`INITs`是表达式。首先会求出`INIT`表达式，之后创建新的帧，将这些值绑定到`VARs`，再然后在新环境中求出`BODY`，返回它的值。换句话说，它等价于调用

```scheme
((lambda (VAR1 VAR2 ...) BODY)
INIT1 INIT2 ...)
```

所以，任何`INIT`表达式中的`VARs`引用都指向这些符号在`let`结构外的定义（如果存在的话），例如：

In [ ]:
(let ((x 2) (y 3))
...       (* x y))

In [ ]:
(let ((x 2) (y 3))
...       (let ((x 7) (z (+ x y)))
...            (* z x)))

`(let* BINDINGS BODY)`

`BINDINGS `的语法和`let`相同。它等价于

In [ ]:
(let ((VAR1 INIT1))
...
(let ((VARn INITn))
BODY))

也就是说，它就像`let`表达式那样，除了`VAR1`的新绑定对`INITs`子序列以及`BODY`中可见，`VAR2`与之类似，例如：

In [ ]:
(define x 3)

In [ ]:
(define y 4)

In [ ]:
(let ((x 5) (y (+ x 1))) y)

In [ ]:
(let* ((x 5) (y (+ x 1))) y)

`(letrec BINDINGS BODY)`

同样，语法类似于`let`。这里，首先会创建新的绑定（带有未定义的值），之后`INITs`被求值并赋给它们。如果某个`INITs`使用了某个`VAR`的值，并且没有为其赋初始值，结果是未定义的。这个形式主要用于定义互相递归的函数（lambda 本身并不会使用它们提到过的值；这只会在它们被调用时随后发生）。例如：

```scheme
(letrec ((even?
      (lambda (n)
             (if (zero? n)
                  #t
                  (odd? (- n 1)))))
     (odd?
      (lambda (n)
              (if (zero? n)
                  #f
                  (even? (- n 1))))))
(even? 88))
```

**内部定义。**当`BODY`以`define`结构的序列开始时，它们被看作“内部定义”，并且在解释上与顶层定义有些不同。特别是，它们就像`letrec`那样。

+ 首先，会为所有由`define`语句定义的名称创建绑定，一开始绑定到未定义的值上。
+ 之后，值由定义来填充。

所以，内部函数定义的序列是互相递归的，就像 `Python 中嵌套在函数中的`def`语句那样：

In [ ]:
(define (hard-even? x)     ;; An outer-level definition
...      (define (even? n)      ;; Inner definition
...          (if (zero? n)
...              #t
...              (odd? (- n 1))))
...      (define (odd? n)       ;; Inner definition
...          (if (zero? n)
...              #f
...              (even? (- n 1))))
...      (even? x))

In [ ]:
(hard-even? 22)

**预定义函数。**预定义函数有很多，都在全局环境中绑定到名称上，我们只会展示一小部分。其余的会在[ Revised(4) Scheme 报告](http://people.csail.mit.edu/jaffer/r4rs_toc.html)中列出。函数调用并不是“特殊的”，因为它们都使用相同的完全统一的求值规则：递归求出所有项目（包括运算符），并且之后在操作数的值上调用运算符的值（它必须是个函数）。

+   **算数：**Scheme 提供了标准的算数运算符，许多都拥有熟悉的表示，虽然它们统一出现在操作数前面：

In [ ]:
; Semicolons introduce one-line comments.

In [ ]:
; Compute (3+7+10)*(1000-8) // 992 - 17

In [ ]:
(- (quotient (* (+ 3 7 10) (- 1000 8))) 17)

In [ ]:
(- 17)

    
    与之相似，存在通用的数学比较运算符，为可接受多于两个参数而扩展：
    

    ```scheme
    >>> (< 0 5)
    #t
    >>> (>= 100 10 10 0)
    #t
    >>> (= 21 (* 7 3) (+ 19 2))
    #t
    >>> (not (= 15 14))
    #t
    >>> (zero? (- 7 7))
    #t
    ```

    
    随便提一下，`not`是个函数，并不是`and`或`or`的特殊形式，因为他的运算符必须求值，所以不需要特殊对待。
    
+   **列表和偶对。**很多操作用于处理偶对和列表（它们同样由偶对和空列表构建）。

    
    ```scheme
    >>> (cons 'a 'b)
    (a . b)
    >>> (list 'a 'b)
    (a b)
    >>> (cons 'a (cons 'b '()))
    (a b)
    >>> (car (cons 'a 'b))
    a
    >>> (cdr (cons 'a 'b))
    b
    >>> (cdr (list a b))
    (b)
    >>> (cadr '(a b))   ; An abbreviation for (car (cdr '(a b)))
    b
    >>> (cddr '(a b))   ; Similarly, an abbreviation for (cdr (cdr '(a b)))
    ()
    >>> (list-tail '(a b c) 0)
    (a b c)
    >>> (list-tail '(a b c) 1)
    (b c)
    >>> (list-ref '(a b c) 0)
    a
    >>> (list-ref '(a b c) 2)
    c
    >>> (append '(a b) '(c d) '() '(e))
    (a b c d e)
    >>> ; All but the last list is copied.  The last is shared, so:
    >>> (define L1 (list 'a 'b 'c))
    >>> (define L2 (list 'd))
    >>> (define L3 (append L1 L2))
    >>> (set-car! L1 1)
    >>> (set-car! L2 2)
    >>> L3
    (a b c 2)
    >>> (null? '())
    #t
    >>> (list? '())
    #t
    >>> (list? '(a b))
    #t
    >>> (list? '(a . b))
    #f
    ```

    
+   **相等性：**`=`运算符用于数值。通常对于值的相等性，Scheme 区分`eq?`（就像 Python 的`is`），`eqv?`（与之类似，但是和数值上的`=`一样），和`equal?`（比较列表结构或字符串的内容）。通常来说，除了在比较符号、布尔值或者空列表的情况中，我们都使用`eqv?`和`equal?`。
    

    ```scheme
    >>> (eqv? 'a 'a)
    #t
    >>> (eqv? 'a 'b)
    #f
    >>> (eqv? 100 (+ 50 50))
    #t
    >>> (eqv? (list 'a 'b) (list 'a 'b))
    #f
    >>> (equal? (list 'a 'b) (list 'a 'b))
    #t
    ```

    
+   **类型。**每个值的类型都只满足一个基本的类型断言。

    ```scheme
    >>> (boolean? #f)
    #t
    >>> (integer? 3)
    #t
    >>> (pair? '(a b))
    #t
    >>> (null? '())
    #t
    >>> (symbol? 'a)
    #t
    >>> (procedure? +)
    #t
    ```

+   **输入和输出：**Scheme 解释器通常执行“读取-求值-打印”循环，但是我们可以在程序控制下显式输出东西，使用与解释器内部相同的函数：

    ```scheme
    >>> (begin (display 'a) (display 'b) (newline))
    ab
    ```

    
    于是，`(display x)`与 Python 的

    
    ```py
    print(str(x), end="")
    ```

    
    相似，并且`(newline)`类似于`print()`。
    
    对于输入来说，`(read)`从当前“端口”读取 Scheme 表达式。它并不会解释表达式，而是将其读作数据：

    
    ```scheme
    >>> (read)
    >>> (a b c)
    (a b c)
    ```

    
+   **求值。**`apply `函数提供了函数调用运算的直接访问：

    ```scheme
    >>> (apply cons '(1 2))
    (1 . 2)
    >>> ;; Apply the function f to the arguments in L after g is
    >>> ;; applied to each of them
    >>> (define (compose-list f g L)
    ...     (apply f (map g L)))
    >>> (compose-list + (lambda (x) (* x x)) '(1 2 3))
    14
    ```

    
    这个扩展允许开头出现“固定”参数：

    
    ```scheme
    >>> (apply + 1 2 '(3 4 5))
    15
    ```

    
    下面的函数并不在 [Revised(4) Scheme](http://people.csail.mit.edu/jaffer/r4rs_toc.html) 中，但是存在于我们的解释器版本中（警告：非标准的过程在 Scheme 的后续版本中并不以这种形式定义）：

    
    ```scheme
    >>> (eval '(+ 1 2))
    3
    ```

    
    也就是说，`eval`求解一块 Scheme 数据，它表示正确的 Scheme 表达式。这个版本在全局环境中求解表达式的参数。我们的解释器也提供了一种方式，来规定求值的特定环境：

    
    ```scheme
    >>> (define (incr n) (lambda (x) (+ n x)))
    >>> (define add5 (incr 5))
    >>> (add5 13)
    18
    >>> (eval 'n (procedure-environment add5))
    5
    ```

## 3.6.2 Logo 语言

Logo 是 Lisp 的另一种方言。它为教育用途而设计，所以 Logo 的许多设计决策是为了让语言对新手更加友好。例如，多数 Logo 过程以前缀形式调用（首先是过程名称，其次是参数），但是通用的算术运算符以普遍的中缀形式提供。Logo 的伟大之处是，它的简单亲切的语法仍旧为高级程序员提供了惊人的表现力。

Logo 的核心概念是，它的内建容器类型，也就是 Logo `sentence `（也叫作列表），可以轻易储存 Logo 源码，这也是它的强大表现力的来源。Logo 的程序可以编写和执行 Logo 表达式，作为求值过程的一部分。许多动态语言都支持代码生成，包括 Python，但是没有语言像 Logo 一样使代码生成如此有趣和易用。

你可能希望下载完整的 Logo 解释器来体验这个语言。标准的实现是 [Berkeley Logo](http://www.cs.berkeley.edu/~bh/logo.html)（也叫做 UCBLogo），由 Brian Harvey 和他的 Berkeley 学生开发。对于苹果用户，[ACSLogo](http://www.alancsmith.co.uk/logo/) 兼容 Mac OSX 的最新版本，并带有一份介绍 Logo 语言许多特性的[用户指南](http://www.alancsmith.co.uk/logo/LogoUserGuide151.pdf)。

**基础。**Logo 设计为会话式。它的读取-求值循环的提示符是一个问号（`?`），产生了“我下面应该做什么？”的问题。我们自然想让它打印数值：

```logo
? print 5
5
```

Logo 语言使用了非标准的调用表达式语法，完全不带括号分隔符。上面，参数`5`转给了`print`，它打印了它的参数。描述 Logo 程序结构的术语有些不同于 Python。Logo 拥有过程而不是 Python 中等价的函数，而且过程输出值而不是返回值。和 python 类似，`print`过程总是输出`None`，但也打印出参数的字符串表示作为副作用。（过程的参数在 Logo 中也通常叫做输入，但是为了清晰起见，这篇文章中我们仍然称之为参数。）

Logo 中最常见的数据类型是单词，它是不带空格的字符串。单词用作可以表示数值、名称和布尔值的通用值。可以解释为数值或布尔值的记号，比如`5`，直接求值为单词。另一方面，类似`five`的名称解释为过程调用：

```logo
? 5
You do not say what to do with 5.
? five
I do not know how to five.
```

`5`和`five`以不同方式解释，Logo 的读取-求值循环也以不同方式报错。第一种情况的问题是，Logo 在顶层表达式不求值为 None 时报错。这里，我们看到了第一个 Logo 不同于计算器的结构；前者的接口是读取-解释循环，期待用户来打印结果。后者使用更加通用的读取-求值-打印循环，自动打印出返回值。Python 采取了混合的方式，非`None`的值使用`repr`强制转换为字符串并自动打印。

Logo 的行可以顺序包含多个表达式。解释器会一次求出每个表达式。如果行中任何顶层表达式不求值为`None`，解释器会报错。一旦发生错误，行中其余的表达式会被忽略。

```logo
? print 1 print 2
1
2
? 3 print 4
You do not say what to do with 3.
```

Logo 的调用表达式可以嵌套。在 Logo 的实现版本中，每个过程接受固定数量的参数。所以，当嵌套调用表达式的操作数完整时，Logo 解释器能够唯一地判断。例如，考虑两个过程`sum`和`difference`，它们相应输出两个参数的和或差。

```logo
? print sum 10 difference 7 3
14
```

我们可以从这个嵌套的例子中看到，分隔调用表达式的圆括号和逗号不是必须的。在计算器解释器中，标点符号允许我们将表达式树构建为纯粹的句法操作，没有任何运算符名称的判断。在 Logo 中，我们必须使用我们的知识，关于每个过程接受多少参数，来得出嵌套表达式的正确结构。下一节中，问题的细节会深入探讨。

Logo 也支持中缀运算符，例如`+`和`*`。这些运算符的优先级根据代数的标准规则来解析。乘法和除法优于加法和减法：

```logo
? 2 + 3 * 4
14
```

如何实现运算符优先级和前缀运算符来生成正确的表达式树的细节留做练习。对于下面的讨论，我们会专注于使用前缀语法的调用表达式。

**引用。**一个名称会被解释为调用表达式的开始部分，但是我们也希望将单词引用为数据。以双引号开始的记号解释为单词字面值。要注意单词字面值在 Logo 中并没有尾后的双引号。

```logo
? print "hello
hello
```

在 Lisp 的方言中（而 Logo 是它的方言），任何不被求值的表达式都叫做引用。这个引用的概念来自于事物之间的经典哲学。例如一只狗，它可以到处乱跑和叫唤，而单词“狗”只是用于指代这种事物的语言结构。当我们以引号使用“狗”的时候，我们并不是指特定的哪一只，而是这个单词。在语言中，引号允许我们谈论语言自身，Logo 中也一样。我们可以按照名称引用`sum`过程，而不会实际调用它，通过这样引用它：

```logo
? print "sum
sum
```

除了单词，Logo 包含句子类型，可以叫做列表。句子由方括号包围。`print`过程并不会打印方括号，以维持 Logo 的惯例风格，但是方括号可以使用`show`过程打印到输出：

```logo
? print [hello world]
hello world
? show [hello world]
[hello world]
```

句子也可以使用三个不同的二元过程来构造。`sentence`过程将参数组装为句子。它是多态过程，如果参数是单词，会将它的参数放入新的句子中；如果参数是句子，则会将拼接参数。结果通常是一个句子：

```logo
? show sentence 1 2
[1 2]
? show sentence 1 [2 3]
[1 2 3]
? show sentence [1 2] 3
[1 2 3]
? show sentence [1 2] [3 4]
[1 2 3 4]
```

`list`过程从两个元素创建句子，它允许用户创建层次数据结构：

```logo
? show list 1 2
[1 2]
? show list 1 [2 3]
[1 [2 3]]
? show list [1 2] 3
[[1 2] 3]
? show list [1 2] [3 4]
[[1 2] [3 4]]
```

最后，`fput`过程从第一个元素和列表的剩余部分创建列表，就像这一章之前的 Python `RList`构造器那样：

```logo
? show fput 1 [2 3]
[1 2 3]
? show fput [1 2] [3 4]
[[1 2] 3 4]
```

我们在 Logo 中可以调用`sentence`、`list`和`fput`句子构造器。在 Logo 中将句子解构为`first`和剩余部分（叫做`butfirst`）也非常直接，所以，我们也拥有一系列句子的选择器过程。

```logo
? print first [1 2 3]
1
? print last [1 2 3]
3
? print butfirst [1 2 3]
[2 3]
```

**作为数据的表达式。**句子的内容可以直接当做未求值的引用。所以，我们可以打印出 Logo 表达式而不求值：

```logo
? show [print sum 1 2]
[print sum 1 2]
```

将 Logo 表示表达式表示为句子的目的通常不是打印它们，而是使用`run`过程来求值。

```logo
? run [print sum 1 2]
3
```

通过组合引用和句子构造器，以及`run`过程，我们获得了一个非常通用的组合手段，它凭空构建 Logo 表达式并对其求值：

```logo
? run sentence "print [sum 1 2]
3
? print run sentence "sum sentence 10 run [difference 7 3]
14
```

最后一个例子的要点是为了展示，虽然`sum`和`difference`过程在 Logo 中并不是一等的构造器（它们不能直接放在句子中），它们的名称是一等的，并且`run`过程可以将它们的名称解析为所引用的过程。

将代码表示为数据，并且稍后将其解释为程序的一部分的功能，是 Lisp 风格语言的特性。程序可以重新编写自己来执行是一个强大的概念，并且作为人工智能（AI）早期研究的基础。Lisp 在数十年间都是 AI 研究者的首选语言。[Lisp 语言](http://www.softwarepreservation.org/projects/LISP/book/LISP%201.5%20Programmers%20Manual.pdf)由 John McCarthy 发明，他也发明了“人工智能”术语，并且在该领域的定义中起到关键作用。Lisp 方言的“代码即数据”的特性，以及它们的简洁和优雅，今天仍继续吸引着 Lisp 程序员。

**海龟制图（Turtle graphics）。**所有 Logo 的实现都基于 Logo 海龟 来完成图形输出。这个海龟以画布的中点开始，基于过程移动和转向，并且在它的轨迹上画线。虽然海龟为鼓励青少年实践编程而发明，它对于高级程序员来说也是有趣的图形化工具。

在执行 Logo 程序的任意时段，Logo 海龟都在画布上拥有位置和朝向。类似于`forward`和`right`的一元过程能修改海龟的位置和朝向。常用的过程都有缩写：`forward`也叫作`fd`，以及其它。下面的嵌套表达式画出了每个端点带有小星星的大星星：

```logo
? repeat 5 [fd 100 repeat 5 [fd 20 rt 144] rt 144]
```

![](img/star.png)

海龟过程的全部指令也内建于 Python 的[`turtle`模块](http://docs.python.org/py3k/library/turtle.html)中。这些函数的有限子集也在这一章的配套项目中提供。

**赋值。**Logo 支持绑定名称和值。就像 Python 中那样，Logo 环境由帧的序列组成，每个帧中的某个名称都最多绑定到一个值上。名称使用`make`过程来绑定，它接受名称和值作为参数。

```logo
? make "x 2
```

任何以冒号起始的单词，例如`:x`都叫做变量。变量求值为其名称在当前环境中绑定的值。

`make`过程和 Python 的赋值语句具有不同的作用。传递给`make`的名称要么已经绑定了值，要么当前未绑定。

1.  如果名称已经绑定，`make`在找到它的第一帧中重新绑定该名称。
2.  如果没有绑定，`make`在全局帧中绑定名称。

这个行为与 Python 赋值语句的语义很不同，后者总是在当前环境中的第一帧中绑定名称。上面的第一条规则类似于遵循`nonlocal`语句的 Python 赋值。第二条类似于遵循`global`语句的全局赋值。

**过程。**Logo 支持用户使用以`to`关键字开始的定义来定义过程。定义是 Logo 中的最后一个表达式类型，在调用表达式、基本表达式和引用表达式之后。定义的第一行提供了新过程的名称，随后是作为变量的形参。下面的行是过程的主体，它可以跨越多行，并且必须以只包含`end`记号的一行结束。Logo 的读取-求值循环使用`>`连接符来提示用户输入过程体。用户定义过程使用`output`过程来输出一个值。

```logo
? to double :x
> output sum :x :x
> end
? print double 4
8
```

Logo 的用户定义过程所产生的调用过程和 Python 中的过程类似。在一系列参数上调用过程以使用新的帧扩展当前环境，以及将过程的形参绑定到实参开始，之后在开始于新帧的环境中求出过程体的代码行。

`output`的调用在 Logo 中与 Python 中的`return`语句有相同作用：它会中止过程体的执行，并返回一个值。Logo 过程可以通过调用`stop`来不带任何值返回。

```logo
? to count
> print 1
> print 2
> stop
> print 3
> end
? count
1
2
```

**作用域。**Logo 是动态作用域语言。类似 Python 的词法作用域语言并不允许一个函数的局部名称影响另一个函数的求值，除非第二个函数显式定义在第一个函数内。两个顶层函数的形参完全是隔离的。在动态作用域的语言中，没有这种隔离。当一个函数调用另一个函数时，绑定到第一个函数局部帧的名称可在第二个函数的函数体中访问：

```logo
? to print_last_x
> print :x
> end
? to print_x :x
> print_last_x
> end
? print_x 5
5
```

虽然名称`x`并没有在全局帧中绑定，而是在`print_x`的局部帧中，也就是首先调用的函数。Logo 的动态作用域规则允许函数`print_last_x`引用`x`，它被绑定到`print_x`的形式参数上。

动态作用域只需要一个对计算环境模型的简单修改就能实现。由用户函数调用创建的帧总是扩展自当前环境（调用处）。例如，上面的`print_x`调用引入了新的帧，它扩展自当前环境，当前环境中包含`print_x`的局部帧和全局帧。所以，在`print_last_x`的主体中查找`x`会发现局部帧中该名称绑定到`5`。与之相似，在 Python 的词法作用域下，`print_last_x`的帧只扩展自全局帧（定义处），而并不扩展自`print_x`的局部帧（调用处）。

动态作用域语言拥有一些好处，它的过程可能不需要接受许多参数。例如，`print_last_x`上面的过程没有接受参数，但是它的行为仍然由内层作用域参数化。

**常规编程。**我们的 Logo 之旅就到此为止了。我们还没有介绍任何高级特性，例如，对象系统、高阶过程，或者语句。学会在 Logo 中高效编程需要将语言的简单特性组合为有效的整体。

Logo 中没有条件表达式类型。过程`if`和`ifelse`使用调用表达式的求值规则。`if`的第一个参数是个布尔单词，`True`或者`False`。第二个参数不是输出值，而是一个句子，包含如果第一个参数为`True`时需要求值的代码行。这个设计的重要结果是，第二个函数的内容如果不被用到就不会全部求值。

```logo
? 1/0
div raised a ZeroDivisionError: division by zero
? to reciprocal :x
> if not :x = 0 [output 1 / :x]
> output "infinity
> end
? print reciprocal 2
0.5
? print reciprocal 0
infinity
```

Logo 的条件语句不仅仅不需要特殊语法，而且它实际上可以使用`word`和`run`实现。`ifelse`的基本过程接受三个函数：布尔单词、如果单词为`True`需要求值的句子，和如果单词为`False`需要求值的句子。通过适当命名形式参数，我们可以实现拥有相同行为的用户定义过程`ifelse2`。
 

```logo
? to ifelse2 :predicate :True :False
> output run run word ": :predicate
> end
? print ifelse2 emptyp [] ["empty] ["full]
empty
```

递归过程不需要任何特殊语法，它们可以和`run`、`sentence`、`first`和`butfirst`一起使用，来定义句子上的通用序列操作。例如，我们可以通过构建二元句子并执行它，来在参数上调用过程。如果参数是个单词，它必须被引用。

```logo
? to apply_fn :fn :arg
> output run list :fn ifelse word? :arg [word "" :arg] [:arg]
> end
```

下面，我们可以定义一个过程，它在句子`:s`上逐步映射函数`:fn`。

```logo
? to map_fn :fn :s
> if emptyp :s [output []]
> output fput apply_fn :fn first :s map_fn :fn butfirst :s
> end
? show map "double [1 2 3]
[2 4 6]
```

`map_fn`主体的第二行也可以使用圆括号编写，表明调用表达式的嵌套结构。但是，圆括号表示了调用表达式的开始和末尾，而不是包围在操作数和非运算符周围。

```logo
> (output (fput (apply_fn :fn (first :s)) (map_fn :fn (butfirst :s))))
```

圆括号在 Logo 中并不必须，但是它们通常帮助程序员记录嵌套表达式的结构。许多 Lisp 的方言都需要圆括号，所以就拥有了显式嵌套的语法。

作为最后一个例子，Logo 可以以非常紧凑的形式使用海龟制图来递归作图。谢尔宾斯基三角是个分形图形，它绘制每个三角形的同时还绘制邻近的三个三角形，它们的顶点是包含它们的三角形的边上的中点。它可以由这个 Logo 程序以有限的递归深度来绘制。

```logo
? to triangle :exp
> repeat 3 [run :exp lt 120]
> end

? to sierpinski :d :k
> triangle [ifelse :k = 1 [fd :d] [leg :d :k]]
> end

? to leg :d :k
> sierpinski :d / 2 :k - 1
> penup fd :d pendown
> end
```

`triangle `过程是个通用方法，它重复三次绘制过程，并在每个重复之后左转。`sierpinski `过程接受长度和递归深度。如果深度为`1`，它画出纯三角形，否则它画出由`log`的调用所组成的三角形。`leg`过程画出谢尔宾斯基递归三角型的一条边，通过递归调用`sierpinski `填充这条边长度的上一半，之后将海龟移动到另一个顶点上。过程`up`和`down`通过将笔拿起并在之后放下，在海龟移动过程中停止画图。`sierpinski`和`leg`之间的多重递归产生了如下结果：

```logo
? sierpinski 400 6
```

![](img/sier.png)

## 3.6.3 结构

这一节描述了 Logo 解释器的通常结构。虽然这一章是独立的，它也确实引用了配套项目。完成这个项目会从零制造出这一章描述的解释器的有效实现。

Logo 的解释器可以拥有和计算器解释器相同的结构。解析器产生表达式数据结构，它们可由求值器来解释。求值函数检查表达式的形式，并且对于调用表达式，它在一些参数上调用函数来应用某个过程。但是，还是存在一些结构上的不同以适应 Logo 的特殊语法。

**行。**Logo 解析器并不读取一行代码，而是读取可能按序包含多个表达式的整行代码。它不返回表达式树，而是返回 Logo 句子。

解析器实际上只做微小的语法分析。特别是，解析工作并不会将调用表达式的运算符和操作数子表达式区分为树的不同枝干。反之，调用表达式的组成部分顺序排列，嵌套调用表达式表示为摊平的记号序列。最终，解析工作并不判断基本表达式，例如数值的类型，因为 Logo 没有丰富的类型系统。反之，每个元素都是单词或句子。

In [ ]:
 parse_line('print sum 10 difference 7 3')

解析器做了很微小的分析，因为 Logo 的动态特性需要求值器解析嵌套表达式的结构。

解析器并不会弄清句子的嵌套结构，句子中的句子表示为 Python 的嵌套列表。

In [ ]:
parse_line('print sentence "this [is a [deep] list]')

`parse_line`的完整实现在配套项目的`logo_parser.py`中。

**求值。**Logo 一次求值一行。求值器的一个框架实现定义在配套项目的`logo.py`中。从`parse_line`返回的句子传给了`eval_line`函数，它求出行中的每个表达式。`eval_line`函数重复调用`logo_eval`，它求出行中的下一个完整的表达式，直到这一行全部求值完毕，之后返回最后一个值。`logo_eval`函数求出单个表达式。

![](img/logo_eval.png)

`logo_eval`函数求出不同形式的表达式：基本、变量、定义、引用和调用表达式，我们已经在上一节中介绍过它们了。Logo 中多元素表达式的形式可以由检查第一个元素来判断。表达式的每个形式都有自己的求值规则。

1.  基本表达式（可以解释为数值、`True`或`False`的单词）求值为自身。
2.  变量在环境中查找。环境会在下一节中详细讨论。
3.  定义处理为特殊情况。用户定义过程也在下一节中详细讨论。
4.  引用表达式求值为引用的文本，它是个去掉前导引号的字符串。句子（表示为 Python 列表）也看做引用，它们求值为自身。
5.  调用表达式在当前环境中查找运算符名称，并且调用绑定到该名称的过程。

下面是`logo_apply`的简单实现。我们去掉了一些错误检查，以专注于我们的讨论。配套项目中有更加健壮的实现。

In [ ]:
def logo_eval(line, env):
        """Evaluate the first expression in a line."""
        token = line.pop()
        if isprimitive(token):
            return token
        elif isvariable(token):
            return env.lookup_variable(variable_name(token))
        elif isdefinition(token):
            return eval_definition(line, env)
        elif isquoted(token):
            return text_of_quotation(token)
        else:
            procedure = env.procedures.get(token, None)
            return apply_procedure(procedure, line, env)

上面的最后情况调用了第二个过程，表达为函数`apply_procedure`。为了调用由运算符记号命名的过程，这个运算符会在当前环境中查找。在上面的定义中，`env`是`Environment `类的实例，会在下一节中描述。`env.procedures`属性是个储存运算符名称和过程之间映射的字典。在 Logo 中，环境拥有单词的这种映射，并且没有局部定义的过程。而且，Logo 为过程名称和变量名称维护分离的映射，叫做分离的命名空间。但是，以这种方式复用名称并不推荐。

**过程调用。**过程调用以调用`apply_procedure`函数开始，它被传入由`logo_apply`查找到的函数，并带有代码的当前行和当前环境。Logo 中过程调用的过程比计算器中的`calc_apply`更加通用。特别是，`apply_procedure`必须检查打算调用的过程，以便在求解`n`个运算符表达式之前，判断它的参数数量`n`。这里我们会看到，为什么 Logo 解析器不能仅仅由语法分析构建表达式树，因为树的结构由过程决定。

`apply_procedure`函数调用`collect_args `函数，它必须重复调用`logo_eval`来求解行中的下`n`个表达式。之后，计算完过程的参数之后，`apply_procedure`调用了`logo_apply`，实际上这个函数在参数上调用过程。下面的调用图示展示了这个过程。

![](img/logo_apply.png)

最终的函数 `logo_apply`接受两种参数：基本过程和用户定义的过程，二者都是`Procedure`的实例。`Procedure`是一个 Python 对象，它拥有过程的名称、参数数量、主体和形式参数作为实例属性。`body`属性可以拥有不同类型。基本过程在 Python 中已经实现，所以它的`body`就是 Python 函数。用户定义的过程（非基本）定义在 Logo 中，所以它的 `body`就是 Logo 代码行的列表。`Procedure`也拥有两个布尔值属性。一个用于表明是否是基本过程，另一个用于表明是否需要访问当前环境。

In [ ]:
class Procedure():
        def __init__(self, name, arg_count, body, isprimitive=False,
                     needs_env=False, formal_params=None):
            self.name = name
            self.arg_count = arg_count
            self.body = body
            self.isprimitive = isprimitive
            self.needs_env = needs_env
            self.formal_params = formal_params

基本过程通过在参数列表上调用主体，并返回它的返回值作为过程输出来应用。

In [ ]:
def logo_apply(proc, args):
        """Apply a Logo procedure to a list of arguments."""
        if proc.isprimitive:
            return proc.body(*args)
        else:
            """Apply a user-defined procedure"""

用户定义过程的主体是代码行的列表，每一行都是 Logo 句子。为了在参数列表上调用过程，我们在新的环境中求出主体的代码行。为了构造这个环境，我们向当前环境中添加新的帧，过程的形式参数在里面绑定到实参上。这个过程的重要结构化抽象是，求出用户定义过程的主体的代码行，需要递归调用`eval_line`。

**求值/应用递归。**实现求值过程的函数，`eval_line `和`logo_eval`，以及实现函数应用过程的函数，`apply_procedure`、`collect_args`和`logo_apply`，是互相递归的。无论何时调用表达式被发现，求值操作都需要调用它。应用操作使用求值来求出实参中的操作数表达式，以及求出用户定义过程的主体。这个互相递归过程的通用结构在解释器中非常常见：求值以应用定义，应用又使用求值定义。

![](img/eval_apply.png)

这个递归循环终止于语言的基本类型。求值的基本条件是，求解基本表达式、变量、引用表达式或定义。函数调用的基本条件是调用基本过程。这个互相递归的结构，在处理表达式形式的求值函数，和处理函数及其参数的应用之间，构成了求值过程的本质。

## 3.6.4 环境

既然我们已经描述了 Logo 解释器的结构，我们转而实现`Environment `类，便于让它使用动态作用域正确支持赋值、过程定义和变量查找。`Environment`实例表示名称绑定的共有集合，可以在程序执行期间的某一点上访问。绑定在帧中组织，而帧以 Python 字典实现。帧包含变量的名称绑定，但不包含过程。运算符名称和`Procedure`实例之间的绑定在 Logo 中是单独储存的。在这个实现中，包含变量名称绑定的帧储存为字典的列表，位于`Environment`的`_frames`属性中，而过程名称绑定储存在值为字典的`procedures`属性中。

帧不能直接访问，而是通过两个`Environment`的方法：`lookup_variable`和`set_variable_value`。前者实现了一个过程，与我们在第一章的计算环境模型中引入的查找过程相同。名称在当前环境第一帧（最新添加）中与值匹配。如果它被找到，所绑定的值会被返回。如果没有找到，会在被当前帧扩展的帧中寻找。

`set_variable_value `也会寻找与变量名称匹配的绑定。如果找到了，它会更新为新的值，如果没有找到，那么会在全局帧上创建新的绑定。这些方法的实现留做配套项目中的练习。

`lookup_variable `方法在求解变量名称时由`logo_eval`调用。`set_variable_value `由`logo_make`函数调用，它用作 Logo 中`make`基本过程的主体。

除了变量和`make`基本过程之外，我们的解释器支持它的第一种抽象手段：将名称绑定到值上。在 Logo 中，我们现在可以重复我们第一章中的第一种抽象步骤。

```logo
? make "radius 10
? print 2 * :radius
20
```

赋值只是抽象的一种有限形式。我们已经从这门课的开始看到，即使对于不是很大的程序，用户定义函数也是管理复杂性的关键工具。我们需要两个改进来实现 Logo 中的用户定义过程。首先，我们必须描述`eval_definition`的实现，如果当前行是定义，`logo_eval`会调用这个 Python 函数。其次，我们需要在`logo_apply`中完成我们的描述，它在一些参数上调用用户过程。这两个改动都需要利用上一节定义的`Procedure`类。

定义通过创建新的`Procedure`实例来求值，它表示用户定义的过程。考虑下面的 Logo 过程定义：

```logo
? to factorial :n
> output ifelse :n = 1 [1] [:n * factorial :n - 1]
> end
? print fact 5
120
```

定义的第一行提供了过程的名称`factorial`和形参`n`。随后的一些行组成了过程体。这些行并不会立即求值，而是为将来使用而储存。也就是说，这些行由`eval_definition`读取并解析，但是并不传递给`eval_line`。主体中的行一直读取，直到出现了只包含`end`的行。在 Logo 中，`end`并不是需要求值的过程，也不是过程体的一部分。它是个函数定义末尾的语法标记。

`Procedure`实例从这个过程的名称、形参列表以及主体中创建，并且在环境中的`procedures`的字典属性中注册。不像 Python，在 Logo 中，一旦过程绑定到一个名称，其它定义都不能复用这个名称。

`logo_apply`将`Procedure`实例应用于一些参数，它是表示为字符串的 Logo 值（对于单词），或列表（对于句子）。对于用户定义过程，`logo_apply`创建了新的帧，它是一个字典对象，键是过程的形参，值是实参。在动态作用域语言例如 Logo 中，这个新的帧总是扩展自过程调用处的当前环境。所以，我们将新创建的帧附加到当前环境上。之后，主体中的每一行都依次传递给`eval_line `。最后，在主体完成求值后，我们可以从环境中移除新创建的帧。由于 Logo 并不支持高阶或一等过程，在程序执行期间，我们并不需要一次跟踪多于一个环境。

下面的例子演示了帧的列表和动态作用域规则，它们由调用这两个 Logo 用户定义过程产生：

```logo
? to f :x
> make "z sum :x :y
> end
? to g :x :y
> f sum :x :x
> end
? g 3 7
? print :z
13
```

由这些表达式的求值创建的环境分为过程和帧，它们维护在分离的命名空间中。帧的顺序由调用顺序决定。

![](img/scope.png)

## 3.6.5 数据即程序

在思考求值 Logo 表达式的程序时，一个类比可能很有帮助。程序含义的一个可取观点是，程序是抽象机器的描述。例如，再次思考下面的计算阶乘的过程：

```logo
? to factorial :n
> output ifelse :n = 1 [1] [:n * factorial :n - 1]
> end
```

我们可以在 Python 中表达为等价的程序，使用传统的表达式。

In [ ]:
def factorial(n):
        return 1 if n == 1 else n * factorial(n - 1)

我们可能将这个程序看做机器的描述，它包含几个部分，减法、乘法和相等性测试，并带有两相开关和另一个阶乘机器（阶乘机器是无限的，因为它在其中包含另一个阶乘机器）。下面的图示是一个阶乘机器的流程图，展示了这些部分是怎么组合到一起的。

![](img/factorial_machine.png)

与之相似，我们可以将 Logo 解释器看做非常特殊的机器，它接受机器的描述作为输入。给定这个输入，解释器就能配置自己来模拟描述的机器。例如，如果我们向解释器中输入阶乘的定义，解释器就可以计算阶乘。

![](img/universal_machine.png)

从这个观点得出，我们的 Logo 解释器可以看做通用的机器。当输入以 Logo 程序描述时，它就能模拟其它机器。它在由我们的编程语言操作的数据对象，和编程语言自身之间起到衔接作用。想象一下，一个用户在我们正在运行的 Logo 解释器中输入了 Logo 表达式。从用户的角度来看，类似`sum 2 2`的输入表达式是编程语言中的表达式，解释器应该对其求值。但是，从解释器的角度来看，表达式只是单词组成的句子，可以根据定义好的一系列规则来操作它。

用户的程序是解释器的数据，这不应该是混乱的原因。实际上，有时候忽略这个差异会更方便，以及让用户能够显式将数据对象求值为表达式。在 Logo 中，无论我们何时使用`run `过程，我们都使用了这种能力。Python 中也存在相似的函数：`eval`函数会求出 Python 表达式，`exec`函数会求出 Python 语句，所以：

In [ ]:
eval('2+2')

和

In [ ]:
2+2

返回了相同的结果。求解构造为指令的一部分的表达式是动态编程语言的常见和强大的特性。这个特性在 Logo 中十分普遍，很少语言是这样，但是在程序执行期间构造和求解表达式的能力，对任何程序员来说都是有价值的工具。